In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d
from scipy.stats import gmean
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
from statistics import mean

pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 15)

## Data upload & interpolation

In [2]:
def cube_data_loading():
       df_cube = pd.read_csv('C:\jupyter\SPP\input\cubeDataBal8_10.csv').drop(columns=['Unnamed: 0'])[['wellName','DEPT', 'DEN_cube_orig_mean_extracted',
              'FORMATION', 'GR_cube_orig_mean_extracted', 'NET', 'PHIT_cube_orig_mean_extracted', 'TST','Vsh_cl_cube_orig_mean_extracted']]
       df_cube = df_cube.rename(columns={'wellName':'well',
                                   'DEPT':'MD',
                                   'DEN_cube_orig_mean_extracted':'RHOB_cube', 
                                   'GR_cube_orig_mean_extracted':'GR_cube', 
                                   'PHIT_cube_orig_mean_extracted':'PHIT_cube', 
                                   'Vsh_cl_cube_orig_mean_extracted':'VSH_cube'})
       df_cube.loc[df_cube.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
       df_cube.loc[df_cube.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
       return df_cube
# df_cube = cube_data_loading()

In [3]:
def well_gr_phit_cube(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    vsh = well_sel['VSH_cube']
    phit = well_sel['PHIT_cube']
    net = well_sel[var]
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=2, subplot_titles=('VSH', 'PHIT'))
    fig.add_trace(go.Scatter(x=vsh, y=depth, mode='lines', name='VSH', 
                             line=dict(color='gray')), row=1, col=1)
    
    fig.update_xaxes(range = [0,1])
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 1], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=700, width=400,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50))
    fig.show()
# well_gr_phit_cube(df_cube, 'Balakhany X','NET', 'A01')

In [4]:
#Loading metadata, distribution wells per Platforms and all the that.
def metadata_parquet_loading():
    path = 'C:\\jupyter\\SPP\\input\\'
    metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
    metadata = metadata_init.copy()
    metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
    metadata.Status = metadata.Status.str.strip()
    metadata.Status = metadata.Status.str.lower()
    metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
    metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
    metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
    metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
    metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
    metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
    metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
    metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
    metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
    metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
    metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
    metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
    metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
    metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
    metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
    metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
    metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
    metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'

    #Read data from parquet
    path = 'C:\\jupyter\\SPP\\input\\'
    df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v10.parquet.gzip')
    df_prq.rename(columns={'wellName':'well'}, inplace=True)
    df_prq = df_prq.set_index('well').join(metadata.set_index('well')).reset_index()
    # Filter data with bad_well_list 
    bad_well_list = ['E10Z','Predrill_J01Z', 'Predrill_J08', 'J28_bpQIP', 'A01W_2']
    df_prq = df_prq[~df_prq.well.isin(bad_well_list)]
    #Assign any Fluidcode_mod number by variable gross_pay=1 and gross_pay=0 if Fluidcode_mod as NaN
    df_prq.loc[df_prq.FLUIDS>0, 'FLUIDS_int'] = 1
    df_prq.loc[df_prq.FLUIDS<=0, 'FLUIDS_int'] = 0
    df_prq.FLUIDS_int = df_prq.FLUIDS_int.astype('int')
    # Unite of FU for each formation
    df_bal = df_prq[df_prq.FORMATION.str.contains('Balakhany')]
    df_bal.loc[df_bal.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
    df_bal.loc[df_bal.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
    df_bal = df_bal[df_bal.FORMATION_up.notna()]
    #Getting XY mean coords of Balakhany formation
    xy_coord_mean = df_bal[['well', 'FORMATION_up', 'X', 'Y']]
    xy_coord_mean = xy_coord_mean.groupby(['well', 'FORMATION_up']).agg({'X': 'mean', 'Y':'mean'}).reset_index()
    xy_coord_mean = xy_coord_mean.rename(columns={'X':'X_mean', 'Y':'Y_mean'})
    xy_coord_mean = xy_coord_mean[xy_coord_mean.FORMATION_up.str.contains('Balakhany') & (xy_coord_mean.X_mean>0) & (xy_coord_mean.Y_mean>0)]
    df_bal.rename(columns={'X':'X_traj', 'Y':'Y_traj'}, inplace=True)
    df_bal = df_bal.set_index(['well', 'FORMATION_up']).join(xy_coord_mean.set_index(['well', 'FORMATION_up'])).reset_index()
    return df_bal
df_bal = metadata_parquet_loading()

In [ ]:
def interpolate_by_depth_fm_run(df, step):
    df_tst = df[df.TST.notna()].round({'MD':1})
    
    def interpolate_by_depth_fm(one_well, step):
        one_well = one_well.sort_values(by='TST')
        well_name = one_well["well"].iloc[0]
        data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
        starting_tst = one_well["TST"].iloc[0]
        new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
        interp_MD = interp1d(one_well['TST'], one_well['MD'], kind='linear', fill_value="extrapolate")
        interp_DEVI = interp1d(one_well['TST'], one_well['DEVI'], kind='linear', fill_value="extrapolate")
        interp_HAZI = interp1d(one_well['TST'], one_well['HAZI'], kind='linear', fill_value="extrapolate")
        interp_NET = interp1d(one_well['TST'], one_well['NET'], kind='linear', fill_value="extrapolate")
        interp_NET_VSH = interp1d(one_well['TST'], one_well['NET_VSH'], kind='linear', fill_value="extrapolate")
        # interp_FLUIDS = interp1d(one_well['TST'], one_well['FLUIDS'], kind='linear', fill_value="extrapolate")
        interp_LPERM = interp1d(one_well['TST'], one_well['LPERM'], kind='linear', fill_value="extrapolate")
        interp_PHIT = interp1d(one_well['TST'], one_well['PHIT'], kind='linear', fill_value="extrapolate")
        interp_GR = interp1d(one_well['TST'], one_well['GR_N'], kind='linear', fill_value="extrapolate")
        interp_VSH = interp1d(one_well['TST'], one_well['VSH'], kind='linear', fill_value="extrapolate")
        interp_NPSS = interp1d(one_well['TST'], one_well['NPSS'], kind='linear', fill_value="extrapolate")
        interp_RHOB = interp1d(one_well['TST'], one_well['RHOB'], kind='linear', fill_value="extrapolate")
        interp_RDEEP = interp1d(one_well['TST'], one_well['RDEEP'], kind='linear', fill_value="extrapolate")
        interp_SON = interp1d(one_well['TST'], one_well['SON'], kind='linear', fill_value="extrapolate")
        interp_SONSH = interp1d(one_well['TST'], one_well['SONSH'], kind='linear', fill_value="extrapolate")
        interp_TVDSCS = interp1d(one_well['TST'], one_well['TVD_SCS'], kind='linear', fill_value="extrapolate")
        interp_Xtraj = interp1d(one_well['TST'], one_well['X_traj'], kind='linear', fill_value="extrapolate")
        interp_Ytraj = interp1d(one_well['TST'], one_well['Y_traj'], kind='linear', fill_value="extrapolate")
        interp_Xmean = interp1d(one_well['TST'], one_well['X_mean'], kind='linear', fill_value="extrapolate")
        interp_Ymean = interp1d(one_well['TST'], one_well['Y_mean'], kind='linear', fill_value="extrapolate")
        interp_RHOF = interp1d(one_well['TST'], one_well['RHOF'], kind='linear', fill_value="extrapolate") 
        interp_RHOMA = interp1d(one_well['TST'], one_well['RHOMA'], kind='linear', fill_value="extrapolate")     
        # Create a new DataFrame with the interpolated values for new TVD_SCS
        new_data = {
            'well': [well_name for _ in range(len(new_TST_values))],
            # 'FORMATION_up': [formation_name for _ in range(len(new_TST_values))],
            'tst_index': [_ for _ in range(len(new_TST_values))],
            'MD': interp_MD(new_TST_values),
            'TST': new_TST_values,
            'DEVI': interp_DEVI(new_TST_values),
            'HAZI': interp_HAZI(new_TST_values),
            'NET': interp_NET(new_TST_values),
            'NET_VSH': interp_NET_VSH(new_TST_values),
            # 'FLUIDS': interp_FLUIDS(new_TST_values),
            'LPERM': interp_LPERM(new_TST_values),
            'PHIT': interp_PHIT(new_TST_values),
            'GR_N': interp_GR(new_TST_values),
            'VSH': interp_VSH(new_TST_values),
            'NPSS': interp_NPSS(new_TST_values),
            'RHOB': interp_RHOB(new_TST_values),
            'RDEEP': interp_RDEEP(new_TST_values),
            'SON': interp_SON(new_TST_values),
            'SONSH': interp_SONSH(new_TST_values),
            'TVD_SCS': interp_TVDSCS(new_TST_values),
            'X_traj': interp_Xtraj(new_TST_values),
            'Y_traj': interp_Ytraj(new_TST_values),
            'Xmean': interp_Xmean(new_TST_values),
            'Ymean':interp_Ymean(new_TST_values),
            'RHOF': interp_RHOF(new_TST_values),
            'RHOMA':interp_RHOMA(new_TST_values)
        }
        new_df = pd.DataFrame(new_data)
        return new_df
    df_lst = []
    print('Start interpolation')
    for wellnames in tqdm(df_tst.well.unique()):
        well_sel = df_tst[df_tst.well == wellnames]
        well_interp = interpolate_by_depth_fm(well_sel, step)
        df_lst.append(well_interp)
    df_interp = pd.concat(df_lst)
    df_interp = df_interp.round({'MD':1, 'TVD_SCS':1, 'TST':1})

    print('Start joining')
    def well_bal_interp_join(dataset):
        df_tst = df[(df.TST.notna()) & (df.FORMATION_up.notna())].round({'MD':1})
        data_fu = df_tst[['well','MD','FORMATION_up', 'FORMATION', 'field']]
        well_join = dataset.set_index(['well','MD']).join(data_fu.set_index(['well','MD'])).reset_index()
        well_join.insert(3, 'FORMATION_up', well_join.pop('FORMATION_up'))
        well_join.insert(4, 'FORMATION', well_join.pop('FORMATION'))
        well_join.insert(5, 'tst_index', well_join.pop('tst_index'))
        return well_join
    well_interp_v2 = well_bal_interp_join(df_interp)
    well_interp_v2.loc[well_interp_v2.NET_VSH > 0, 'NET_VSH'] = 1
    well_interp_v2.loc[well_interp_v2.NET > 0, 'NET'] = 1
    return well_interp_v2
df_interp_old = interpolate_by_depth_fm_run(df_bal, 0.1)
df_interp_old = df_interp_old[df_interp_old.well != 'B28']
df_interp_old = df_interp_old[df_interp_old.well != 'B27']

In [ ]:
# 'C27X', 'C27Y','J09Z','J13X','C19Z' - wells with absence of X,Y trajectories
def interpolate_by_depth_new_wells():
    df_new = pd.read_parquet('C:\jupyter\SPP\input\JOIN_BEST_new130324.parquet')
    df_new = df_new.replace(-9999.000, 0)

    xy_coord_mean = df_new.groupby(['well', 'FORMATION_up']).agg({'X_traj': 'mean', 'Y_traj':'mean'}).reset_index()
    xy_coord_mean = xy_coord_mean.rename(columns={'X_traj':'X_mean', 'Y_traj':'Y_mean'})
    xy_coord_mean = xy_coord_mean[xy_coord_mean.FORMATION_up.str.contains('Balakhany') & (xy_coord_mean.X_mean>0) & (xy_coord_mean.Y_mean>0)]
    df_new = df_new.set_index(['well', 'FORMATION_up']).join(xy_coord_mean.set_index(['well', 'FORMATION_up'])).reset_index()
    df_new['field'] = 'new_wells'

    def new_well_detection(df_new, df_old):
        new_well = []
        for well_new in df_new.well.unique():
            if well_new in df_old.well.unique():
                pass
            else:
                new_well.append(well_new)
        return new_well
    new_well = new_well_detection(df_new, df_interp_old)
    new_well.append('B28')

    gr8_well_list = ['E20Y', 'E10Z', 'D19Y', 'C27X', 'C19Z', 'B28', 'A01']
    df_new.loc[(df_new.well.isin(gr8_well_list))  & (df_new.FORMATION_up == 'Balakhany VIII') 
                                                & (df_new.GR_N <= 60), 'NET'] = 1
    gr10_well_list = ['A01', 'B28', 'D19Y', 'E20Y', 'J09Z']
    df_new.loc[(df_new.well.isin(gr10_well_list))  & (df_new.FORMATION_up == 'Balakhany X') 
                                                & (df_new.GR_N <= 60), 'NET'] = 1

    def interpolate_by_depth_fm_run(df, step):
        df_tst = df[df.TST.notna()].round({'MD':1})
        
        def interpolate_by_depth_fm(one_well, step):
            one_well = one_well.sort_values(by='TST')
            well_name = one_well["well"].iloc[0]
            data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
            starting_tst = one_well["TST"].iloc[0]
            new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
            interp_MD = interp1d(one_well['TST'], one_well['MD'], kind='linear', fill_value="extrapolate")
            interp_DEVI = interp1d(one_well['TST'], one_well['DEVI'], kind='linear', fill_value="extrapolate")
            interp_HAZI = interp1d(one_well['TST'], one_well['HAZI'], kind='linear', fill_value="extrapolate")
            interp_NET = interp1d(one_well['TST'], one_well['NET'], kind='linear', fill_value="extrapolate")
            interp_NET_VSH = interp1d(one_well['TST'], one_well['NET_VSH'], kind='linear', fill_value="extrapolate")
            # interp_FLUIDS = interp1d(one_well['TST'], one_well['FLUIDS'], kind='linear', fill_value="extrapolate")
            interp_LPERM = interp1d(one_well['TST'], one_well['LPERM'], kind='linear', fill_value="extrapolate")
            interp_PHIT = interp1d(one_well['TST'], one_well['PHIT'], kind='linear', fill_value="extrapolate")
            interp_GR = interp1d(one_well['TST'], one_well['GR_N'], kind='linear', fill_value="extrapolate")
            interp_VSH = interp1d(one_well['TST'], one_well['VSH'], kind='linear', fill_value="extrapolate")
            interp_NPSS = interp1d(one_well['TST'], one_well['NPSS'], kind='linear', fill_value="extrapolate")
            interp_RHOB = interp1d(one_well['TST'], one_well['RHOB'], kind='linear', fill_value="extrapolate")
            interp_RDEEP = interp1d(one_well['TST'], one_well['RDEEP'], kind='linear', fill_value="extrapolate")
            interp_SON = interp1d(one_well['TST'], one_well['SON'], kind='linear', fill_value="extrapolate")
            interp_SONSH = interp1d(one_well['TST'], one_well['SONSH'], kind='linear', fill_value="extrapolate")
            interp_TVDSCS = interp1d(one_well['TST'], one_well['TVD_SCS'], kind='linear', fill_value="extrapolate")
            interp_Xtraj = interp1d(one_well['TST'], one_well['X_traj'], kind='linear', fill_value="extrapolate")
            interp_Ytraj = interp1d(one_well['TST'], one_well['Y_traj'], kind='linear', fill_value="extrapolate")
            interp_Xmean = interp1d(one_well['TST'], one_well['X_mean'], kind='linear', fill_value="extrapolate")
            interp_Ymean = interp1d(one_well['TST'], one_well['Y_mean'], kind='linear', fill_value="extrapolate")
            interp_RHOF = interp1d(one_well['TST'], one_well['RHOF'], kind='linear', fill_value="extrapolate") 
            interp_RHOMA = interp1d(one_well['TST'], one_well['RHOMA'], kind='linear', fill_value="extrapolate")     
            # Create a new DataFrame with the interpolated values for new TVD_SCS
            new_data = {
                'well': [well_name for _ in range(len(new_TST_values))],
                # 'FORMATION_up': [formation_name for _ in range(len(new_TST_values))],
                'tst_index': [_ for _ in range(len(new_TST_values))],
                'MD': interp_MD(new_TST_values),
                'TST': new_TST_values,
                'DEVI': interp_DEVI(new_TST_values),
                'HAZI': interp_HAZI(new_TST_values),
                'NET': interp_NET(new_TST_values),
                'NET_VSH': interp_NET_VSH(new_TST_values),
                # 'FLUIDS': interp_FLUIDS(new_TST_values),
                'LPERM': interp_LPERM(new_TST_values),
                'PHIT': interp_PHIT(new_TST_values),
                'GR_N': interp_GR(new_TST_values),
                'VSH': interp_VSH(new_TST_values),
                'NPSS': interp_NPSS(new_TST_values),
                'RHOB': interp_RHOB(new_TST_values),
                'RDEEP': interp_RDEEP(new_TST_values),
                'SON': interp_SON(new_TST_values),
                'SONSH': interp_SONSH(new_TST_values),
                'TVD_SCS': interp_TVDSCS(new_TST_values),
                'X_traj': interp_Xtraj(new_TST_values),
                'Y_traj': interp_Ytraj(new_TST_values),
                'Xmean': interp_Xmean(new_TST_values),
                'Ymean':interp_Ymean(new_TST_values),
                'RHOF': interp_RHOF(new_TST_values),
                'RHOMA':interp_RHOMA(new_TST_values)
            }
            new_df = pd.DataFrame(new_data)
            return new_df
        df_lst = []
        print('Start interpolation')
        for wellnames in tqdm(df_tst.well.unique()):
            well_sel = df_tst[df_tst.well == wellnames]
            well_interp = interpolate_by_depth_fm(well_sel, step)
            df_lst.append(well_interp)
        df_interp = pd.concat(df_lst)
        df_interp = df_interp.round({'MD':1, 'TVD_SCS':1, 'TST':1})

        print('Start joining')
        def well_bal_interp_join(dataset):
            df_tst = df[(df.TST.notna()) & (df.FORMATION_up.notna())].round({'MD':1})
            data_fu = df_tst[['well','MD','FORMATION_up', 'FORMATION', 'field']]
            well_join = dataset.set_index(['well','MD']).join(data_fu.set_index(['well','MD'])).reset_index()
            well_join.insert(3, 'FORMATION_up', well_join.pop('FORMATION_up'))
            well_join.insert(4, 'FORMATION', well_join.pop('FORMATION'))
            well_join.insert(5, 'tst_index', well_join.pop('tst_index'))
            return well_join
        well_interp_v2 = well_bal_interp_join(df_interp)
        well_interp_v2.loc[well_interp_v2.NET_VSH > 0, 'NET_VSH'] = 1
        well_interp_v2.loc[well_interp_v2.NET > 0, 'NET'] = 1
        return well_interp_v2
    df_new_interp = interpolate_by_depth_fm_run(df_new, 0.1)

    def perm_ds_us_calc(dataset):
        def perm_ds_1(x):
            return 10**(39139.46 * pow(x, 5) - 11140.04 * pow(x, 4) + 855.2176 * pow(x, 3) + 7.04505 * pow(x, 2) + 5.750233 * x - 1.997085)
        def perm_us_1(x):
            return 10**(1638286 * pow(x, 5) - 1396883 * pow(x, 4) + 468324.8 * pow(x, 3) - 76974.79 * pow(x, 2) + 6217.262 * x - 198.5042)
        def perm_ds_2(x):
            return 10**(+ 5675.143 * pow(x, 5) - 11106.91 * pow(x, 4) + 8608.366 * pow(x, 3) - 3318.893 * pow(x, 2) + 644.7713 * x - 48.16968)
        def perm_us_2(x):
            return 10**(+ 81.59968 * pow(x, 5) - 275.5442 * pow(x, 4) + 364.9522 * pow(x, 3) - 238.8838 * pow(x, 2) + 79.97139 * x - 7.15)
        df = dataset
        try:
            df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = df.PHIT.apply(perm_ds_1)
            df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = df.PHIT.apply(perm_us_1)
            df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_DS'] = df.PHIT.apply(perm_ds_2)
            df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_US'] = df.PHIT.apply(perm_us_2)
        except:
            df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = 0
            df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = 0
            df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_DS'] = 0
            df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_US'] = 0
        df['PERM_DS'] = df['PERM_DS'].fillna(0)
        df['PERM_US'] = df['PERM_US'].fillna(0)
        return df

    df_new_well8 = df_new_interp[df_new_interp.well.isin(new_well) & (df_new_interp.FORMATION_up == 'Balakhany VIII') & (~df_new_interp.well.isin(['E12Z']))]
    # df_new_well8 = perm_ds_us_calc(df_new_well8)
    df_new_well10 = df_new_interp[df_new_interp.well.isin(new_well) & (df_new_interp.FORMATION_up == 'Balakhany X') & (~df_new_interp.well.isin(['A01','C27X']))]
    # df_new_well10 = perm_ds_us_calc(df_new_well10)
    df_new_interp2 = pd.concat([df_new_well8, df_new_well10])
    return df_new_interp2
df_interp_new = interpolate_by_depth_new_wells()
df_interp_new['NET'] = df_interp_new['NET'].fillna(0)
df_interp_new['V'] = ((df_interp_new.GR_N - 39) / (87-39)).clip(0,1)
df_interp_new['VSH'] = ((1.7 - (3.38 - (df_interp_new.V + 0.7)**2)**0.5)).clip(0,1)

def xy_uploading_corr_run(dataset_base):
    def xy_new_uploading():
        xy_new = pd.read_csv('C:\\jupyter\\SPP\\input\\slb_xy_JOIN_BEST.csv')[1:]
        xy_new = xy_new.rename(columns={'wellName':'well'})
        xy_new = xy_new.replace(-9999.0, np.nan)
        xy_new = xy_new.replace('-9999', np.nan)
        xy_new = xy_new[['well', 'X', 'Y', 'FORMATION']].dropna(subset=['FORMATION','X','Y'])
        xy_new.loc[xy_new.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
        xy_new.loc[xy_new.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
        xy_new = xy_new[['well', 'X', 'Y', 'FORMATION', 'FORMATION_up']].dropna(subset=['FORMATION_up'])
        xy_new_gb = xy_new.groupby(['well','FORMATION_up']).agg({'X':'mean', 'Y':'mean'}).reset_index().rename(columns={'X':'Xmean', 'Y':'Ymean'})
        return xy_new_gb
    xy_new_gb = xy_new_uploading()

    def xy_new_correction(dataset_base, dataset_add, fm, wellname):
        dataset_base.loc[(dataset_base.well == wellname) & 
                        (dataset_base.FORMATION_up == fm), 
                        'Xmean'] = dataset_add[(dataset_add.well == wellname) & 
                                            (dataset_add.FORMATION_up == fm)]['Xmean'].values[0]
        dataset_base.loc[(dataset_base.well == wellname) & 
                        (dataset_base.FORMATION_up == fm), 
                        'Ymean'] = dataset_add[(dataset_add.well == wellname) & 
                                            (dataset_add.FORMATION_up == fm)]['Ymean'].values[0]
        return dataset_base
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany VIII', 'C19Z')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany VIII', 'C27Y')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany X', 'C27Y')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany X', 'J09Z')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany VIII', 'J13X')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany X', 'J13X')
    return df_interp_new
df_interp_new = xy_uploading_corr_run(df_interp_new)

df_interp = pd.concat([df_interp_old, df_interp_new])

In [ ]:
df_interp_new.well.unique()

In [57]:
def check_presence_gr_cube_techlog():
    df_cube8 = df_cube[df_cube.FORMATION_up == 'Balakhany VIII']
    df_cube10 = df_cube[df_cube.FORMATION_up == 'Balakhany X']
    df_interp8 = df_interp[df_interp.FORMATION_up == 'Balakhany VIII']
    def check_presence_well(dataset, dataset_cube):
        well_list = []
        for well in dataset.well.unique():
            if well in dataset_cube.well.unique():
                pass
            else:
                well_list.append(well)  
        return well_list
    well_list8 = check_presence_well(df_interp8, df_cube8)
    well_list10 = check_presence_well(df_interp, df_cube10)
    well_list8_r = check_presence_well(df_cube8, df_interp8)
    well_list10_r = check_presence_well(df_cube10, df_interp)
    print('well_list8',np.array(well_list8))
    print('well_list10', np.array(well_list10))
    print('well_list8_r', np.array(well_list8_r))
    print('well_list10_r', np.array(well_list10_r))
# check_presence_gr_cube_techlog()

In [60]:
def vsh_phit_cleaning(df, cutoff):
    md_total = df.groupby(['well','FORMATION_up'])['MD'].count().reset_index()
    df_phit = df[df.PHIT > 0]
    phit_total = df_phit.groupby(['well','FORMATION_up'])['PHIT'].count().reset_index()
    df_vsh = df[df.VSH > 0]
    vsh_total = df_vsh.groupby(['well','FORMATION_up'])['VSH'].count().reset_index()
    
    full_df = md_total.set_index(['well','FORMATION_up']).join(phit_total.set_index(['well','FORMATION_up'])).reset_index()
    full_df_v2 = full_df.set_index(['well','FORMATION_up']).join(vsh_total.set_index(['well','FORMATION_up'])).reset_index()
    full_df_v2['phit_md'] = full_df_v2['PHIT'] / full_df_v2['MD']
    full_df_v2['vsh_md'] = full_df_v2['VSH'] / full_df_v2['MD']
    full_df_v2 = full_df_v2.round({'phit_md':2, 'vsh_md':2})

    phit_clean_bal8 = full_df_v2[(full_df_v2['phit_md'] >= cutoff) & (full_df_v2.FORMATION_up == 'Balakhany VIII')].reset_index(drop=True)
    wells_porostiy_back = ['B04', 'B29', 'B31', 'B40', 'C01AY', 'C08Z', 'C38', 'D04Z', 'D29']
    phit_well_back_bal8 = full_df_v2[full_df_v2.well.isin(wells_porostiy_back)]
    phit_clean_bal8 = pd.concat([phit_clean_bal8, phit_well_back_bal8])
    phit_clean_bal10 = full_df_v2[(full_df_v2['phit_md'] >= cutoff) & (full_df_v2.FORMATION_up == 'Balakhany X')].reset_index(drop=True)
    vsh_clean_bal8 = full_df_v2[(full_df_v2['vsh_md'] >= cutoff) & (full_df_v2.FORMATION_up == 'Balakhany VIII')].reset_index(drop=True)
    vsh_clean_bal10 = full_df_v2[(full_df_v2['vsh_md'] >= cutoff) & (full_df_v2.FORMATION_up == 'Balakhany X')].reset_index(drop=True)
    result = {'phit_bal8': phit_clean_bal8, 'phit_bal10':phit_clean_bal10, 'vsh_bal8': vsh_clean_bal8, 'vsh_bal10':vsh_clean_bal10}
    return result
vsh_clean_bal8 = vsh_phit_cleaning(df_interp, 0.9)['vsh_bal8']
phit_clean_bal8 = vsh_phit_cleaning(df_interp, 0.9)['phit_bal8']
vsh_clean_bal10 = vsh_phit_cleaning(df_interp, 0.9)['vsh_bal10']
phit_clean_bal10 = vsh_phit_cleaning(df_interp, 0.9)['phit_bal10']

In [9]:
# df_interp_test = df_interp[(df_interp.FORMATION_up == 'Balakhany VIII') & (df_interp.FORMATION != 'Balakhany VIII')]
# phit_clean_bal8_test = vsh_phit_cleaning(df_interp_test, 0.9)['phit_bal8']
# for wellname in phit_clean_bal8_test.well.unique():
#     if wellname not in phit_clean_bal8.well.unique():
#         print('1st:', wellname)

# for wellname in phit_clean_bal8.well.unique():
#     if wellname not in phit_clean_bal8_test.well.unique():
#         print('2d:',wellname)

## NET cleaning for Bal8 Azeri

### netvsh_clp2 bal8

In [61]:
def well_display_logs(dataset, wellname, fmname, var1, var2, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    data_net_khtst = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname) & 
                             ((dataset.FORMATION != 'Balakhany VIII') & (dataset.FORMATION != 'Balakhany X'))]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data[var1]*100
    net_init_calc = data_net_khtst[var1]
    net = data[var2]*100
    net_calc = data_net_khtst[var2]
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    khtst = data['KHtst']

    var1_sum = (net_init_calc.sum()*0.1).round(0)
    var2_sum = (net_calc.sum()*0.1).round(0)
    tst_interval = round(depth.max() - depth.min(),0)
    height = int(tst_interval/15)
    khtst_top = int(data_net_khtst['KHtst'].iloc[0])

    fig, ax = plt.subplots(1,4, figsize=(7,height), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # ax[0].set_title((var1 + ':' + str(var1_sum) + 'm'), fontsize=10)
    ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    # ax[2].set_title((var2 + ':' + str(var2_sum) + 'm'), fontsize=10)
    ax[2].set_title(var2 + ':' + str(var2_sum), fontsize=10)
    # ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth, net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    twin4 = ax[3].twiny()
    twin4.plot(khtst, depth, color='black', alpha=1)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' 
                + ref_depth +':' + str(tst_interval) + 
                ' KHtst:' + str(khtst_top), fontsize=12)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png', dpi=300)
    else:
        pass
    plt.show();
# well_display_logs(df_bal8_azr_v3, 'C16', 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal8\\')

In [62]:
def netvsh_calc_by_gr8(df_interp):
    df_interp.loc[(df_interp.well == 'B17')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B30')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B35')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B36')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B38Y')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'C27Z')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    return df_interp
df_interp = netvsh_calc_by_gr8(df_interp)

def vsh_calc_by_gr8(df_interp): 
    gr8_well_list = ['B04Z', 'B12Z', 'B18X', 'B18Y', 'B22Z', 'B25', 'B27', 'B28', 'B33',
                    'C03', 'C05', 'C07Z', 'C08', 'C11Z', 'C13', 'C15Z', 'C19', 'C22',
                    'C23', 'C24', 'C25Z', 'C29', 'C30', 'C34Z', 'D01Y', 'D05Y', 'D10',
                    'D14Z', 'D16', 'D17', 'D17Z', 'D18', 'D19', 'D20X', 'D20Z', 'D22',
                    'D23Y', 'D23Z', 'D28']
    df_interp.loc[(df_interp.well.isin(gr8_well_list))  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                                        & (df_interp.VSH <= 0.5), 'NET'] = 1
    return df_interp
df_interp = vsh_calc_by_gr8(df_interp)

def well_gr_corr_display(dataset, formation, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']*100
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', marker=dict(color='green')))
    fig.update_xaxes(range=[0, 150])
    fig.update_yaxes(autorange="reversed")
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  marker=dict(color='rgba(245, 190, 20, 0.75)'), fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'))
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=700, width=250, title_text=well, showlegend=False,
                    margin=dict(l=10,r=10,b=10,t=40))
    fig.show()
# well_gr_corr_display(df_interp, 'Balakhany VIII', 'B12Z')

In [ ]:
def net_flag_cleaning(dataset_main, fm):
    df_net_bal = dataset_main

    print('Calculation dataframe with h_tst from MD to TST')
    def ntd_calculation_brief(dataset,well, desired_fm, net_var):
        data = dataset[(dataset.well==well)]
        data['TST'].iloc[0] = 0
        data['TST'].iloc[-1] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_list = []
    for well in tqdm(df_net_bal.well.unique()):
        df = ntd_calculation_brief(df_net_bal, well, fm, 'NET')
        df_list.append(df)
    ntd_net = pd.concat(df_list)
    # ntd_net

    # Cleaning NET variable and making up NET_clp with clipped data, join NET_clp to main dataframe
    print('Cleaning NET variable and making up NET_clp with clipped data')
    def ntd_htst_cleaning(dataset_main, dataset_ntd, cutoff):
        ntd_well = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_cutoff = ntd_well[ntd_well.h_tst >= cutoff]
        well_short = dataset_main[['well', 'FORMATION', 'FORMATION_up', 'MD', 'TST', 'GR_N', 'NET', ]]
        net_well = well_short[(well_short.well==well)]
        net_well['NET_clp'] = 0
        for j in range(len(ntd_well_cutoff.well)):
            ntd_top = ntd_well_cutoff.iloc[j, 2].round(3)
            ntd_bot = ntd_well_cutoff.iloc[j, 3].round(3)
            for i in range(len(net_well.TST)):
                well_tst = net_well['TST'].iloc[i].round(3)
                if well_tst >= ntd_top and well_tst <= ntd_bot:
                    net_well['NET_clp'].iloc[i] = 1
        return net_well
    df_list_ntd = []
    for well in tqdm(ntd_net.well.unique()):
        net_well = ntd_htst_cleaning(df_net_bal, ntd_net, 1)
        df_list_ntd.append(net_well)
    net_clp = pd.concat(df_list_ntd)
    # net_clp

    # Cleaning NET_clp from 1-point zero
    print('Cleaning NET_clp from 1-point zero')
    for i in tqdm(range(len(net_clp.NET_clp)-1)):
        if (net_clp.NET_clp.iloc[i] == 0 and  
            net_clp.NET_clp.iloc[i-1] == 1 and 
            net_clp.NET_clp.iloc[i+1] == 1):
            net_clp.NET_clp.iloc[i] = 1        
    df_zero_bal = net_clp[['well', 'MD', 'TST', 'NET_clp', 'FORMATION_up']]
    # df_zero_bal

    print('NET-zero layers searching')
    def ntd_calculation_zero(dataset,well,formation, net_var):
        data = dataset[(dataset.well==well)]
        data.iloc[0, 3] = 1
        data.iloc[-1, 3] = 1
        tst_zero_top = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:]
                    if (data.iloc[i][net_var] == 0 and data.iloc[i-1][net_var] == 1)]
        tst_zero_bot = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:] 
                    if (data.iloc[i][net_var] == 0 and data.iloc[i+1][net_var] == 1)]
        tops_zero = zip(tst_zero_top, tst_zero_bot)
        df_zero_htst = pd.DataFrame(tops_zero, columns=['tst_zero_top', 'tst_zero_bot'])
        df_zero_htst['FORMATION_up'] = formation
        df_zero_htst['well'] = well
        df_zero_htst['h_tst_zero'] = df_zero_htst.tst_zero_bot - df_zero_htst.tst_zero_top
        df_zero_htst = df_zero_htst[['well','FORMATION_up','tst_zero_top','tst_zero_bot','h_tst_zero']]
        return df_zero_htst
    df_zero_list = []
    for well in tqdm(df_zero_bal.well.unique()):
        df = ntd_calculation_zero(df_zero_bal, well, fm, 'NET_clp')
        df_zero_list.append(df)
    ntd_zero = pd.concat(df_zero_list)
    # ntd_zero

    print('NET-zero layers removing')
    def ntd_htst_zero_cleaning(dataset_main, dataset_ntd, cutoff, net_var1, net_var2):
        ntd_well_zero = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_zero_sel = ntd_well_zero[ntd_well_zero.h_tst_zero <= cutoff]
        well_zero_short = dataset_main[['well','FORMATION_up','MD','TST', net_var1, 'GR_N', 'NET', 'FORMATION']]
        well_zero_short[net_var2] = well_zero_short[net_var1]
        well_zero_sel = well_zero_short[(well_zero_short.well==well)]
        for j in range(len(ntd_well_zero_sel.well)):
            ntd_zero_top = ntd_well_zero_sel.iloc[j, 2].round(3)
            ntd_zero_bot = ntd_well_zero_sel.iloc[j, 3].round(3)
            for i in range(len(well_zero_sel.TST)):
                well_zero_tst = well_zero_sel['TST'].iloc[i].round(3)
                if well_zero_tst >= ntd_zero_top and well_zero_tst <= ntd_zero_bot:
                    well_zero_sel[net_var2].iloc[i] = 1
        return well_zero_sel
    df_list_ntd_zero = []
    for well in tqdm(ntd_zero.well.unique()):
        well_zero_sel = ntd_htst_zero_cleaning(net_clp, ntd_zero, 0.75, 'NET_clp', 'NET_clp2')
        df_list_ntd_zero.append(well_zero_sel)
    net_clp2 = pd.concat(df_list_ntd_zero)
    return net_clp2
black_list_wells8 = ['AZERI4', 'AZERI5', 'CHIRAG1', 'CHIRAG4', 'CHIRAG5', 'CHIRAG6', 
                    'GCA1','GCA2', 'GCA4', 'GCA4Z', 'GCA5', 'GCA5Z', 'GCA6', 'GCA6Y', 'GCA6Z','GCA7',
                    'B04','D02']
df_bal8_azr_vsh = df_interp[(df_interp.well.isin(vsh_clean_bal8.well.unique()))
                            & (~df_interp.well.isin(black_list_wells8)) 
                            & (df_interp.FORMATION_up == 'Balakhany VIII')
                            & (df_interp.field.isin(['WEST AZERI', 'CENTRAL AZERI', 'EAST AZERI','new_wells']))]
netvsh_clp2_bal8 = net_flag_cleaning(df_bal8_azr_vsh, 'Balakhany VIII')

def vsh_bal8_net_corr(test_net_clp2):
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 1982.3) 
                                                & (test_net_clp2.TST <= 1986.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B32')  & (test_net_clp2.TST >= 2346) 
                                                    & (test_net_clp2.TST <= 2349), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 147) 
                                                & (test_net_clp2.TST <= 190), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 148.9) 
                                                    & (test_net_clp2.TST <= 156.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 159.1) 
                                                    & (test_net_clp2.TST <= 162.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 165.8) 
                                                    & (test_net_clp2.TST <= 174), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 180.1) 
                                                    & (test_net_clp2.TST <= 189), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1986) 
                                                    & (test_net_clp2.TST <= 1988), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1988) 
                                                    & (test_net_clp2.TST <= 1990), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1990) 
                                                    & (test_net_clp2.TST <= 1995), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1965) 
                                                    & (test_net_clp2.TST <= 1970), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C13Z')  & (test_net_clp2.TST >= 280) 
                                                    & (test_net_clp2.TST <= 284), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C14Z')  & (test_net_clp2.TST >= 123) 
                                                    & (test_net_clp2.TST <= 128), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14Z')  & (test_net_clp2.TST >= 133) 
                                                    & (test_net_clp2.TST <= 135), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14Z')  & (test_net_clp2.TST >= 177) 
                                                    & (test_net_clp2.TST <= 180.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1363) 
                                                    & (test_net_clp2.TST <= 1366), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1437) 
                                                    & (test_net_clp2.TST <= 1452), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1397) 
                                                    & (test_net_clp2.TST <= 1399), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1543) 
                                                    & (test_net_clp2.TST <= 1545), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1620) 
                                                    & (test_net_clp2.TST <= 1625), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1555) 
                                                    & (test_net_clp2.TST <= 1557), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1593.5) 
                                                    & (test_net_clp2.TST <= 1597.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1440) 
                                                    & (test_net_clp2.TST <= 1450), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1525.5) 
                                                    & (test_net_clp2.TST <= 1528), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1528) 
                                                    & (test_net_clp2.TST <= 1531.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 1937) 
                                                    & (test_net_clp2.TST <= 1939), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 2023) 
                                                    & (test_net_clp2.TST <= 2031.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2050) 
                                                    & (test_net_clp2.TST <= 2056), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2109) 
                                                    & (test_net_clp2.TST <= 2111), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2136) 
                                                    & (test_net_clp2.TST <= 2141), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D05ST1') & (test_net_clp2.TST >= 2025) 
                                                    & (test_net_clp2.TST <= 2031), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D09Z') & (test_net_clp2.TST >= 849) 
                                                    & (test_net_clp2.TST <= 855), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D09Z') & (test_net_clp2.TST >= 837.5) 
                                                    & (test_net_clp2.TST <= 843), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y') & (test_net_clp2.TST >= 295.5) 
                                                    & (test_net_clp2.TST <= 298), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y') & (test_net_clp2.TST >= 246.5) 
                                                    & (test_net_clp2.TST <= 247.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D15') & (test_net_clp2.TST >= 3110) 
                                                    & (test_net_clp2.TST <= 3113), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3151.4) 
                                                & (test_net_clp2.TST <= 3156.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3147) 
                                                    & (test_net_clp2.TST <= 3153), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3160) 
                                                & (test_net_clp2.TST <= 3165), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D23X') & (test_net_clp2.TST >= 456) 
                                                    & (test_net_clp2.TST <= 458), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2900) 
                                                    & (test_net_clp2.TST <= 2902), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2918) 
                                                    & (test_net_clp2.TST <= 2922), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2945) 
                                                    & (test_net_clp2.TST <= 2947), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2948) 
                                                    & (test_net_clp2.TST <= 2951), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B06')  & (test_net_clp2.TST >= 1891) 
                                                & (test_net_clp2.TST <= 1897), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B11')  & (test_net_clp2.TST >= 1910) 
                                                    & (test_net_clp2.TST <= 1920), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B14Z')  & (test_net_clp2.TST >= 1688) 
                                                    & (test_net_clp2.TST <= 1690), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B17')  & (test_net_clp2.TST >= 1860) 
                                                    & (test_net_clp2.TST <= 1865), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 1951) 
                                                    & (test_net_clp2.TST <= 1960), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 1994.5) 
                                                    & (test_net_clp2.TST <= 1996.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2053.5) 
                                                    & (test_net_clp2.TST <= 2056.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B29')  & (test_net_clp2.TST >= 2141.5) 
                                                    & (test_net_clp2.TST <= 2146.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B29')  & (test_net_clp2.TST >= 2171) 
                                                    & (test_net_clp2.TST <= 2178), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1850) 
                                                    & (test_net_clp2.TST <= 1865), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1981) 
                                                    & (test_net_clp2.TST <= 1986), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B35')  & (test_net_clp2.TST >= 2329) 
                                                    & (test_net_clp2.TST <= 2337), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B36')  & (test_net_clp2.TST >= -23) 
                                                    & (test_net_clp2.TST <= -19.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B39')  & (test_net_clp2.TST >= 680) 
                                                    & (test_net_clp2.TST <= 690), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B40')  & (test_net_clp2.TST >= 106) 
                                                    & (test_net_clp2.TST <= 116), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B41')  & (test_net_clp2.TST >= 310) 
                                                    & (test_net_clp2.TST <= 314), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B41')  & (test_net_clp2.TST >= 318.5) 
                                                    & (test_net_clp2.TST <= 322.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C03Z')  & (test_net_clp2.TST >= 680) 
                                                    & (test_net_clp2.TST <= 690), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C07')  & (test_net_clp2.TST >= 2036.5) 
                                                    & (test_net_clp2.TST <= 2040), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14')  & (test_net_clp2.TST >= 1960) 
                                                    & (test_net_clp2.TST <= 1970), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C14')  & (test_net_clp2.TST >= 2067) 
                                                    & (test_net_clp2.TST <= 2072), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14')  & (test_net_clp2.TST >= 2033) 
                                                    & (test_net_clp2.TST <= 2043), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C28')  & (test_net_clp2.TST >= 1755) 
                                                    & (test_net_clp2.TST <= 1760), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C28')  & (test_net_clp2.TST >= 1822) 
                                                    & (test_net_clp2.TST <= 1833), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C29Y')  & (test_net_clp2.TST >= 440) 
                                                    & (test_net_clp2.TST <= 450), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1604) 
                                                    & (test_net_clp2.TST <= 1614.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1608) 
                                                    & (test_net_clp2.TST <= 1610), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1614.5) 
                                                    & (test_net_clp2.TST <= 1617), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1620) 
                                                    & (test_net_clp2.TST <= 1622), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1644) 
                                                    & (test_net_clp2.TST <= 1650), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 160) 
                                                & (test_net_clp2.TST <= 170), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 190) 
                                                    & (test_net_clp2.TST <= 200), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 207) 
                                                    & (test_net_clp2.TST <= 210), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 211) 
                                                    & (test_net_clp2.TST <= 218.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C36')  & (test_net_clp2.TST >= 2029) 
                                                    & (test_net_clp2.TST <= 2035), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C36')  & (test_net_clp2.TST >= 2150.5) 
                                                    & (test_net_clp2.TST <= 2160.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C37Z')  & (test_net_clp2.TST >= 813.5) 
                                                    & (test_net_clp2.TST <= 818.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C37Z')  & (test_net_clp2.TST >= 820) 
                                                    & (test_net_clp2.TST <= 824.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C38')  & (test_net_clp2.TST >= 2099.5) 
                                                    & (test_net_clp2.TST <= 2104), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C38')  & (test_net_clp2.TST >= 2106) 
                                                    & (test_net_clp2.TST <= 2109.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C38')  & (test_net_clp2.TST >= 2113.8) 
                                                    & (test_net_clp2.TST <= 2119.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D02')  & (test_net_clp2.TST >= 2931.5) 
                                                    & (test_net_clp2.TST <= 2934), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D02Y')  & (test_net_clp2.TST >= 424.7) 
                                                    & (test_net_clp2.TST <= 426.4), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D03')  & (test_net_clp2.TST >= 2913) 
                                                    & (test_net_clp2.TST <= 2915), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 231) 
                                                    & (test_net_clp2.TST <= 233), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D21')  & (test_net_clp2.TST >= 2922.5) 
                                                    & (test_net_clp2.TST <= 2946.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D21')  & (test_net_clp2.TST >= 2995.7) 
                                                    & (test_net_clp2.TST <= 2998.4), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D32')  & (test_net_clp2.TST >= 2976.5) 
                                                    & (test_net_clp2.TST <= 2979.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D35')  & (test_net_clp2.TST >= 2991.3) 
                                                    & (test_net_clp2.TST <= 3001.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D35')  & (test_net_clp2.TST >= 3019.5) 
                                                    & (test_net_clp2.TST <= 3023), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2877.9) 
                                                    & (test_net_clp2.TST <= 2882.4), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2896.5) 
                                                    & (test_net_clp2.TST <= 2905.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D38')  & (test_net_clp2.TST >= 2366.2) 
                                                    & (test_net_clp2.TST <= 2372.3), 'NET_clp2'] = 1
    return test_net_clp2
netvsh_clp2_bal8 = vsh_bal8_net_corr(netvsh_clp2_bal8)

def phit_bal8_net_corr(test_net_clp2): 
    test_net_clp2.loc[(test_net_clp2.well == 'B02')  & (test_net_clp2.TST >= 1920) 
                                                    & (test_net_clp2.TST <= 1930), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B02')  & (test_net_clp2.TST >= 1960) 
                                                    & (test_net_clp2.TST <= 1965), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B02Z')  & (test_net_clp2.TST >= 357.5) 
                                                    & (test_net_clp2.TST <= 363.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B02Z')  & (test_net_clp2.TST >= 372.5) 
                                                    & (test_net_clp2.TST <= 375.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B02Z')  & (test_net_clp2.TST >= 441.8) 
                                                    & (test_net_clp2.TST <= 445.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B05')  & (test_net_clp2.TST >= 1930) 
                                                    & (test_net_clp2.TST <= 1932.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 1850.5) 
                                                    & (test_net_clp2.TST <= 1852.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 1893) 
                                                    & (test_net_clp2.TST <= 1898), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B08')  & (test_net_clp2.TST >= 1950) 
                                                    & (test_net_clp2.TST <= 1975), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B08')  & (test_net_clp2.TST >= 1959.4) 
                                                    & (test_net_clp2.TST <= 1963.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B08Z')  & (test_net_clp2.TST >= 2754.8) 
                                                    & (test_net_clp2.TST <= 2758.6), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1816.5) 
                                                    & (test_net_clp2.TST <= 1823), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1827.5) 
                                                    & (test_net_clp2.TST <= 1831), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1892.7) 
                                                    & (test_net_clp2.TST <= 1900), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1910.8) 
                                                    & (test_net_clp2.TST <= 1913.2), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B11')  & (test_net_clp2.TST >= 2015.8) 
                                                    & (test_net_clp2.TST <= 2022.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B16Z')  & (test_net_clp2.TST >= 1920.5) 
                                                    & (test_net_clp2.TST <= 1925), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B20')  & (test_net_clp2.TST >= 1963) 
                                                    & (test_net_clp2.TST <= 1964.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2045) 
                                                    & (test_net_clp2.TST <= 2057), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2038.6) 
                                                    & (test_net_clp2.TST <= 2042.8), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2047.2) 
                                                    & (test_net_clp2.TST <= 2050.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B23Z')  & (test_net_clp2.TST >= 1961) 
                                                    & (test_net_clp2.TST <= 1969), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B26')  & (test_net_clp2.TST >= 2110.2) 
                                                    & (test_net_clp2.TST <= 2111.2), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B26')  & (test_net_clp2.TST >= 2113.5) 
                                                    & (test_net_clp2.TST <= 2115.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 118.5) 
                                                    & (test_net_clp2.TST <= 123.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 172.5) 
                                                    & (test_net_clp2.TST <= 187.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B29')  & (test_net_clp2.TST >= 2231) 
                                                    & (test_net_clp2.TST <= 2235.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B30')  & (test_net_clp2.TST >= 1836.7) 
                                                    & (test_net_clp2.TST <= 1843.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1924) 
                                                    & (test_net_clp2.TST <= 1934), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1961.5) 
                                                    & (test_net_clp2.TST <= 1966.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B34')  & (test_net_clp2.TST >= 2285.5) 
                                                & (test_net_clp2.TST <= 2294.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B34Z')  & (test_net_clp2.TST >= 675.9) 
                                                    & (test_net_clp2.TST <= 683.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B35')  & (test_net_clp2.TST >= 2377) 
                                                    & (test_net_clp2.TST <= 2382), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B37')  & (test_net_clp2.TST >= 118.3) 
                                                    & (test_net_clp2.TST <= 121), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B37')  & (test_net_clp2.TST >= 155.5) 
                                                    & (test_net_clp2.TST <= 157), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B39')  & (test_net_clp2.TST >= 803.7) 
                                                    & (test_net_clp2.TST <= 810), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B40')  & (test_net_clp2.TST >= 164.4) 
                                                    & (test_net_clp2.TST <= 167.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C01')  & (test_net_clp2.TST >= 2016.1) 
                                                    & (test_net_clp2.TST <= 2017.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 1914.3) 
                                                    & (test_net_clp2.TST <= 1918.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 1914.3) 
                                                    & (test_net_clp2.TST <= 1918.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 1919.4) 
                                                    & (test_net_clp2.TST <= 1920.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C06')  & (test_net_clp2.TST >= 2049) 
                                                    & (test_net_clp2.TST <= 2056), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C07')  & (test_net_clp2.TST >= 2035.3) 
                                                    & (test_net_clp2.TST <= 2041.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 2021) 
                                                    & (test_net_clp2.TST <= 2022), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 2078.4) 
                                                    & (test_net_clp2.TST <= 2080), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2081) 
                                                    & (test_net_clp2.TST <= 2097), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2090) 
                                                    & (test_net_clp2.TST <= 2091), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C12')  & (test_net_clp2.TST >= 1876.8) 
                                                    & (test_net_clp2.TST <= 1877.8), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C12')  & (test_net_clp2.TST >= 1890) 
                                                    & (test_net_clp2.TST <= 1891.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C12')  & (test_net_clp2.TST >= 1899.3) 
                                                    & (test_net_clp2.TST <= 1901.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C13Z')  & (test_net_clp2.TST >= 278.3) 
                                                    & (test_net_clp2.TST <= 285.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1369.3) 
                                                    & (test_net_clp2.TST <= 1372.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1385.5) 
                                                    & (test_net_clp2.TST <= 1405), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1435) 
                                                    & (test_net_clp2.TST <= 1440.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C20Z')  & (test_net_clp2.TST >= 1578) 
                                                    & (test_net_clp2.TST <= 1582.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1529) 
                                                    & (test_net_clp2.TST <= 1535), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21Z')  & (test_net_clp2.TST >= 312) 
                                                    & (test_net_clp2.TST <= 323), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21Z')  & (test_net_clp2.TST >= 367) 
                                                    & (test_net_clp2.TST <= 371.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C29Y')  & (test_net_clp2.TST >= 441.7) 
                                                    & (test_net_clp2.TST <= 442.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C29Y')  & (test_net_clp2.TST >= 456.3) 
                                                    & (test_net_clp2.TST <= 460), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 1991) 
                                                    & (test_net_clp2.TST <= 1997), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 189.5) 
                                                    & (test_net_clp2.TST <= 192.9), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C34')  & (test_net_clp2.TST >= 2053.1) 
                                                    & (test_net_clp2.TST <= 2058.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C40')  & (test_net_clp2.TST >= 1891.2) 
                                                    & (test_net_clp2.TST <= 1894.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C40')  & (test_net_clp2.TST >= 1935.2) 
                                                    & (test_net_clp2.TST <= 1938.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C41')  & (test_net_clp2.TST >= 2218.2) 
                                                    & (test_net_clp2.TST <= 2222.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C41')  & (test_net_clp2.TST >= 2252.1) 
                                                    & (test_net_clp2.TST <= 2253.4), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C41')  & (test_net_clp2.TST >= 2288) 
                                                    & (test_net_clp2.TST <= 2292), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2133) 
                                                    & (test_net_clp2.TST <= 2144), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2180) 
                                                    & (test_net_clp2.TST <= 2190), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D02Z')  & (test_net_clp2.TST >= 274.4) 
                                                    & (test_net_clp2.TST <= 277), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D03Z')  & (test_net_clp2.TST >= 295) 
                                                    & (test_net_clp2.TST <= 301), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D03Z')  & (test_net_clp2.TST >= 343) 
                                                    & (test_net_clp2.TST <= 352), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D04Z')  & (test_net_clp2.TST >= 755) 
                                                    & (test_net_clp2.TST <= 760.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D04Z')  & (test_net_clp2.TST >= 768.5) 
                                                    & (test_net_clp2.TST <= 771.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D07')  & (test_net_clp2.TST >= 2962) 
                                                    & (test_net_clp2.TST <= 2966), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D12Z')  & (test_net_clp2.TST >= 2120) 
                                                    & (test_net_clp2.TST <= 2130), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D12Z')  & (test_net_clp2.TST >= 2219.4) 
                                                    & (test_net_clp2.TST <= 2224.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D13')  & (test_net_clp2.TST >= 3063) 
                                                    & (test_net_clp2.TST <= 3070), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 231) 
                                                    & (test_net_clp2.TST <= 250), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 290) 
                                                    & (test_net_clp2.TST <= 301.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 303.7) 
                                                    & (test_net_clp2.TST <= 311.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3118) 
                                                    & (test_net_clp2.TST <= 3127.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2942.5) 
                                                    & (test_net_clp2.TST <= 2953.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D25')  & (test_net_clp2.TST >= 2942.5) 
                                                    & (test_net_clp2.TST <= 2953.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D27')  & (test_net_clp2.TST >= 2158) 
                                                    & (test_net_clp2.TST <= 2163), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D27')  & (test_net_clp2.TST >= 2209) 
                                                    & (test_net_clp2.TST <= 2213.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D33')  & (test_net_clp2.TST >= 2986.6) 
                                                    & (test_net_clp2.TST <= 2989.9), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2882.4) 
                                                    & (test_net_clp2.TST <= 2884.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2894.7) 
                                                    & (test_net_clp2.TST <= 2901.1), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D37')  & (test_net_clp2.TST >= 2915) 
                                                    & (test_net_clp2.TST <= 2920), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D38')  & (test_net_clp2.TST >= 2441) 
                                                    & (test_net_clp2.TST <= 2454), 'NET_clp2'] = 1
    return test_net_clp2
netvsh_clp2_bal8 = phit_bal8_net_corr(netvsh_clp2_bal8)

df_bal8_azr_vshclp2 = df_bal8_azr_vsh.set_index(['well','TST']).join(
                      netvsh_clp2_bal8.drop(['FORMATION_up','MD','GR_N','NET','FORMATION'], axis=1).set_index(
                                            ['well','TST'])).reset_index()
df_bal8_azr_vshclp2.loc[df_bal8_azr_vshclp2.well.isin(phit_clean_bal8.well.unique()), 'phit_flag'] = 1
df_bal8_azr_vshclp2['phit_flag'] = df_bal8_azr_vshclp2['phit_flag'].fillna(0)

def perm_ds_us_calc(dataset):
    def perm_ds_1(x):
        return 10**(39139.46 * pow(x, 5) - 11140.04 * pow(x, 4) + 855.2176 * pow(x, 3) + 7.04505 * pow(x, 2) + 5.750233 * x - 1.997085)
    def perm_us_1(x):
        return 10**(1638286 * pow(x, 5) - 1396883 * pow(x, 4) + 468324.8 * pow(x, 3) - 76974.79 * pow(x, 2) + 6217.262 * x - 198.5042)
    def perm_ds_2(x):
        return 10**(+ 5675.143 * pow(x, 5) - 11106.91 * pow(x, 4) + 8608.366 * pow(x, 3) - 3318.893 * pow(x, 2) + 644.7713 * x - 48.16968)
    def perm_us_2(x):
        return 10**(+ 81.59968 * pow(x, 5) - 275.5442 * pow(x, 4) + 364.9522 * pow(x, 3) - 238.8838 * pow(x, 2) + 79.97139 * x - 7.15)
    df = dataset.copy()
    df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = df.PHIT.apply(perm_ds_1)
    df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = df.PHIT.apply(perm_us_1)
    df.loc[(df.PHIT >= 0.2) & (df.PHIT <=1), 'PERM_DS'] = df.PHIT.apply(perm_ds_2)
    df.loc[(df.PHIT >= 0.2) & (df.PHIT <=1), 'PERM_US'] = df.PHIT.apply(perm_us_2)
    df['PERM_DS'] = df['PERM_DS'].fillna(0)
    df['PERM_US'] = df['PERM_US'].fillna(0)
    return df
df_bal8_azr_vshclp2 = perm_ds_us_calc(df_bal8_azr_vshclp2)

def khtst_calculation(dataset):
    df_lst = []
    for wellname in tqdm(dataset.well.unique()):
        well = dataset[dataset.well == wellname]
        well.loc[well['NET'] == 0, 'LPERM'] = 0
        well['k_htst'] = well['LPERM']*0.1
        well.sort_values(by='MD', inplace=True)
        well['KHtst'] = well.k_htst[::-1].cumsum()
        df_lst.append(well)
    well_full = pd.concat(df_lst)
    return well_full
df_bal8_azr_vshclp2 = khtst_calculation(df_bal8_azr_vshclp2)

df_bal8_azr_vshclp2.to_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_vshclp2.csv')

# B14Z, B16Z, B19, B20, C03Z, C10, C11, C21, C32, D12Z, D20 - корректирока NET по VSH после совещания с ВР
def well_gr_smooth_display(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['VSH_smooth']
    # grn = well_sel['VSH']
    phit = well_sel['PHIT']
    net = well_sel[var]
    net_smooth = well_sel['NET_smooth']*100
    # net_smooth = net
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=2, subplot_titles=('VSH', 'PHIT'))
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='VSH', 
                             line=dict(color='gray')), row=1, col=1)
    
    fig.update_xaxes(range = [0,1])
    fig.add_trace(go.Scatter(x=net_smooth, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 1], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=900, width=400,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50))
    fig.show()
well_list = ['B14Z', 'B16Z', 'B19', 'B20', 'B21','B22Z','C03Z', 'C10', 'C11', 'C21', 'C32', 'D12Z', 'D17', 'D20']
percentage = 0.2
def well_gaussian_filter_run(percentage, well_list):
    well_smooth_list = well_list
    smooth_gr = df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well.isin(well_smooth_list)]
    print('Run Gaussian filter for wells')
    def well_gaussian_filter(dataset, wellname, fmname, var, percentage):
        data_full = dataset[dataset.well == wellname]  
        data = dataset[dataset.well == wellname][var]
        coeff = percentage/100
        sigma = int(round(len(data)*coeff, 0))
        data = data.reset_index().drop('index', axis=1)
        smoothed_data = gaussian_filter(data, sigma=sigma)
        data_full[var + '_smooth'] = smoothed_data
        return data_full
    df_lst = []
    for wellname in smooth_gr.well.unique():
        smooth_data = well_gaussian_filter(smooth_gr, wellname, 'Balakhany VIII', 'VSH', percentage)
        df_lst.append(smooth_data)
    vsh_smooth = pd.concat(df_lst)
    vsh_smooth['NET_smooth'] = 0
    vsh_smooth.loc[vsh_smooth.VSH_smooth <= 0.5, 'NET_smooth'] = 1
    return vsh_smooth
vsh_smooth = well_gaussian_filter_run(percentage, well_list)
drop_smooth_wells = df_bal8_azr_vshclp2[~df_bal8_azr_vshclp2.well.isin(well_list)]
add_smooth_wells = pd.concat([drop_smooth_wells, vsh_smooth]).sort_values(by=['well', 'MD'])
df_bal8_azr_v2 = add_smooth_wells.copy()
df_bal8_azr_v2.loc[df_bal8_azr_v2['NET_smooth'].notna(), 'NET_orig'] = df_bal8_azr_v2['NET']
df_bal8_azr_v2.loc[df_bal8_azr_v2['NET_smooth'].notna(), 'NET'] = df_bal8_azr_v2['NET_smooth']
df_bal8_azr_v2.loc[df_bal8_azr_v2['NET_smooth'].notna(), 'NET_VSH_orig'] = df_bal8_azr_v2['NET_VSH']
df_bal8_azr_v2.loc[df_bal8_azr_v2['NET_smooth'].notna(), 'NET_VSH'] = df_bal8_azr_v2['NET_smooth']

In [ ]:
def net_correction(df_bal8_azr_v2):
    vsh_cuttoff = 0.35
    df_bal8_azr_v2.loc[ (df_bal8_azr_v2.well=='C34Z'), 'NET'] = 0
    df_bal8_azr_v2.loc[ (df_bal8_azr_v2.well=='C34Z') &
                        (df_bal8_azr_v2.VSH <= vsh_cuttoff), 'NET'] = 1
    df_bal8_azr_v2.loc[ (df_bal8_azr_v2.well=='C34Z'), 'NET_VSH'] = 0
    df_bal8_azr_v2.loc[ (df_bal8_azr_v2.well=='C34Z') &
                        (df_bal8_azr_v2.VSH <= vsh_cuttoff), 'NET_VSH'] = 1
    return df_bal8_azr_v2
df_bal8_azr_v3 = net_correction(df_bal8_azr_v2)

df_bal8_azr_v3.loc[df_bal8_azr_v3.well.isin(['B27', 'B28', 'B44']), 'field'] = 'CENTRAL AZERI'
df_bal8_azr_v3.loc[df_bal8_azr_v3.well.isin(['C27Y','C19Z', 'C27X']), 'field'] = 'WEST AZERI'
df_bal8_azr_v3.loc[df_bal8_azr_v3.well.isin(['D09Y','D19Z','D19Y']), 'field'] = 'EAST AZERI'
df_bal8_azr_v3 = df_bal8_azr_v3[~df_bal8_azr_v3.well.isin(['C18X','D20Y','E10Z', 'E20Y', 'J13X'])]

df_bal8_azr_v3 = khtst_calculation(df_bal8_azr_v3)

df_bal8_azr_v3 = df_bal8_azr_v3[df_bal8_azr_v3.FORMATION != 'Balakhany VIII']

tst_interv = df_bal8_azr_v3.groupby('well')['TST'].apply(lambda x: x.iloc[-1] - x.iloc[0])
df_bal8_azr_v3 = df_bal8_azr_v3.merge(tst_interv, on='well', suffixes=('', '_interv'))

tst_interv_fu = df_bal8_azr_v3.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[-1] - x.iloc[0])
df_bal8_azr_v3 = df_bal8_azr_v3.merge(tst_interv_fu, on=['well','FORMATION'], suffixes=('', '_interv_fu'))

fluid_code = pd.read_csv(r'C:\jupyter\SPP\input\fluidcode_sql_tl.csv', names = ['well', 'MD','fluid_code'])
fluid_code = fluid_code.astype({'well':'string'})
fluid_code['fluid_code'] = fluid_code['fluid_code'].fillna(0)
fluid_code = fluid_code.drop_duplicates(subset=['well', 'MD'])
df_bal8_azr_v3_fluid = df_bal8_azr_v3.set_index(['well','MD']).join(fluid_code.set_index(['well','MD'])).reset_index().drop(columns=['V'], axis=1)

calcite8 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v2_calc.csv').drop(columns=['Unnamed: 0'])
calcite8 = calcite8[['well','TST','calc', 'calc_net']]
df_bal8_azr_v4 = df_bal8_azr_v3_fluid.set_index(['well','TST']).join(calcite8.set_index(['well','TST'])).reset_index()

df_bal8_azr_v4['phitd_npss'] = (0.6 - df_bal8_azr_v4['NPSS'])*100 - (df_bal8_azr_v4['RHOB'] - 1.67)*60

def interpolate_by_depth_fm_run_v2(df, step):
    df_tst = df[df.TST.notna()].round({'MD':1})
    
    def interpolate_by_depth_fm_v2(one_well, step):
        one_well = one_well.sort_values(by='TST')
        well_name = one_well["well"].iloc[0]
        formation = one_well["FORMATION"].iloc[0]
        formation_up = one_well["FORMATION_up"].iloc[0]
        field = one_well["field"].iloc[0]
        data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
        starting_tst = one_well["TST"].iloc[0]
        new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
        col_lst = []
        for col in one_well.columns:
            if col not in ['well','FORMATION_up', 'FORMATION','field']:
                interp = interp1d(one_well['TST'], one_well[col], kind='linear', fill_value="extrapolate")
                new_data = {col: interp(new_TST_values)}
                new_df = pd.DataFrame(new_data)
                col_lst.append(new_df)
        new_df = pd.concat(col_lst, axis=1)
        new_df['well'] = well_name
        new_df['TST'] = new_TST_values
        new_df['FORMATION'] = formation
        new_df['FORMATION_up'] = formation_up
        new_df['field'] = field #Index(['FORMATION', 'FORMATION_up', 'field'], dtype='object')
        new_df = new_df[[   'well', 
                            'TST', 'tst_index', 'MD',  'DEVI', 'HAZI', 'NET', 'NET_VSH', 'LPERM',
                            'PHIT', 'GR_N', 'VSH', 'NPSS', 'RHOB', 'RDEEP', 'SON', 'SONSH',
                            'TVD_SCS', 'X_traj', 'Y_traj', 'Xmean', 'Ymean', 'RHOF', 'RHOMA',
                            'tst_sample', 'NET_clp', 'NET_clp2', 'phit_flag', 'PERM_DS', 'PERM_US',
                            'k_htst', 'KHtst', 'VSH_smooth', 'NET_smooth', 'NET_orig',
                            'NET_VSH_orig', 'TST_interv', 'TST_interv_fu', 'fluid_code', 'calc',
                            'calc_net', 'phitd_npss']]
        return new_df
    df_lst = []
    for well in tqdm(df_tst.well.unique()):
        well_data = df_tst[df_tst.well == well]
        well_data_interp = interpolate_by_depth_fm_v2(well_data, 0.1)
        df_lst.append(well_data_interp)
    df_interp = pd.concat(df_lst)
    df_interp = df_interp.round({'MD':1, 'TVD_SCS':1, 'TST':1})
    print('Start joining')
    def well_bal_interp_join(dataset):
        df_tst = df[(df.TST.notna()) & (df.FORMATION_up.notna())].round({'MD':1})
        data_fu = df_tst[['well','MD','FORMATION_up', 'FORMATION', 'field']]
        well_join = dataset.set_index(['well','MD']).join(data_fu.set_index(['well','MD'])).reset_index()
        well_join.insert(3, 'FORMATION_up', well_join.pop('FORMATION_up'))
        well_join.insert(4, 'FORMATION', well_join.pop('FORMATION'))
        well_join.insert(5, 'tst_index', well_join.pop('tst_index'))
        return well_join
    well_interp_v2 = well_bal_interp_join(df_interp)
    well_interp_v2.loc[well_interp_v2.NET_VSH > 0, 'NET_VSH'] = 1
    well_interp_v2.loc[well_interp_v2.NET > 0, 'NET'] = 1
    
    df_lst_2 = []
    for well in well_interp_v2.well.unique():
        field_data = well_interp_v2[well_interp_v2.well == well]
        field_data.field = field_data.field.fillna(method = 'ffill')
        field_data.field = field_data.field.fillna(method = 'bfill')
        field_data.FORMATION_up = field_data.FORMATION_up.fillna(method = 'ffill')
        field_data.FORMATION_up = field_data.FORMATION_up.fillna(method = 'bfill')
        field_data.FORMATION = field_data.FORMATION.fillna(method = 'ffill')
        field_data.FORMATION = field_data.FORMATION.fillna(method = 'bfill')
        df_lst_2.append(field_data)
    well_interp_v3 = pd.concat(df_lst_2)

    return well_interp_v3

df_bal8_azr_v4 = interpolate_by_depth_fm_run_v2(df_bal8_azr_v4, 0.1)

df_bal8_azr_v4.to_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v4.csv', index=False)

In [174]:
# df_bal8_azr_v4['tst_sample'] = df_bal8_azr_v4['TST'] - df_bal8_azr_v4['TST'].shift(1)
# df_bal8_azr_v4.tst_sample = df_bal8_azr_v4.tst_sample.round(1)
# df_lst = []
# for wellname in tqdm(df_bal8_azr_v4.well.unique()):
#     test = df_bal8_azr_v4[df_bal8_azr_v4.well == wellname].tst_sample.value_counts().reset_index()
#     test['well'] = wellname
#     df_lst.append(test)
# test2 = pd.concat(df_lst)
# test2[(test2.tst_sample != 0.1) & (test2.tst_sample > 0) ]

In [ ]:
df_bal8_azr_v4 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v4.csv')
df_bal8_azr_v4 = df_bal8_azr_v4[df_bal8_azr_v4.fluid_code.isin([1,2,3])]
custom_palette = {1:'red', 2:'green', 3:'blue'}
sns.histplot(df_bal8_azr_v4[(df_bal8_azr_v4.PHIT != 0) & (df_bal8_azr_v4.NET == 1)], x='PHIT',bins=50, hue='fluid_code', kde=True, palette = custom_palette)

In [16]:
# sns.histplot(data=df_bal8_azr_v4[df_bal8_azr_v4.phitd_npss > 0], 
#              x='phitd_npss', hue='FLUID', bins=50, palette='bright', multiple='stack')

In [30]:
def well_display_logs(dataset, wellname, fmname, var1, var2, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    data_net_khtst = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname) & 
                             ((dataset.FORMATION != 'Balakhany VIII') & (dataset.FORMATION != 'Balakhany X'))]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data[var1]*100
    net_init_calc = data_net_khtst[var1]
    net = data[var2]*100
    net_calc = data_net_khtst[var2]
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    khtst = data['KHtst']

    var1_sum = (net_init_calc.sum()*0.1).round(0)
    var2_sum = (net_calc.sum()*0.1).round(0)
    tst_interval = round(depth.max() - depth.min(),0)
    height = int(tst_interval/15)
    khtst_top = int(data_net_khtst['KHtst'].iloc[0])

    fig, ax = plt.subplots(1,4, figsize=(7,height), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # ax[0].set_title((var1 + ':' + str(var1_sum) + 'm'), fontsize=10)
    ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    # ax[2].set_title((var2 + ':' + str(var2_sum) + 'm'), fontsize=10)
    ax[2].set_title(var2 + ':' + str(var2_sum), fontsize=10)
    # ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth, net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    twin4 = ax[3].twiny()
    twin4.plot(khtst, depth, color='black', alpha=1)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' 
                + ref_depth +':' + str(tst_interval) + 
                ' KHtst:' + str(khtst_top), fontsize=12)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png', dpi=300)
    else:
        pass
    plt.show();
# for wellname in df_bal8_azr_v3.well.unique():
#     print(wellname)
#     well_display_logs(df_bal8_azr_v3, wellname, 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal8\\')


In [31]:
# ['B04Z', 'B12Z', 'B18X', 'B18Y', 'B22Z', 'B25', 'B27', 'B28', 'B33', 'C03', 'C05', 'C07Z', 'C08', 'C11Z', 'C13', 'C15Z', 'C19', 'C22',
#  'C23', 'C24', 'C25Z', 'C29', 'C30', 'C34Z', 'D01Y', 'D05Y', 'D10', 'D14Z', 'D16', 'D17', 'D17Z', 'D18', 'D19', 'D20X', 'D20Z', 'D22',
#  'D23Y', 'D23Z', 'D28']

def well_gr_phit_corr_display(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    phit = well_sel['PHIT']
    rdeep = well_sel['RDEEP']
    net = well_sel[var]*100
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=3, subplot_titles=('GR', 'PHIT', 'RDEEP'))
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', 
                             line=dict(color='green')), row=1, col=1)
    
    fig.update_xaxes(range = [0,150])
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 150], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    fig.add_trace(go.Scatter(x=rdeep, y=depth, mode='lines', name='RDEEP', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=3)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=3)
    fig.update_xaxes(range = [0, 2], type='log', row=1, col=3)
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=850, width=500,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50),
                      xaxis2=dict(  title='xaxis2 title',
                                    overlaying='x2',
                                    side='right'))
    fig.show()
# well_gr_phit_corr_display(df_bal8_azr_v4, 'Balakhany VIII','NET', 'B21')

In [32]:
def well_display_net(dataset, well, formation, net1, net2, depth_step, print_flag, path):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']
    net_clp = well_sel[net1]
    net_clp2 = well_sel[net2]
    
    fig, ax = plt.subplots(1,3, figsize=(6,8), sharey=True)
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(net, depth, color='orange', lw=0.25)
    ax[0].fill_betweenx(depth,net, color='orange', alpha=0.33)
    ax[0].invert_yaxis()
    ax[0].set_xlim(0, 1) 
    ax[0].set_title('NET')
    well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
        ax[0].text(0.1, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
    twin0 = ax[0].twiny()
    twin0.plot(grn, depth, color='green')
    twin0.set_xlim(0, 150)
    twin0.set_xticks([])
    # ax[1].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[1].plot(net_clp, depth, color='orange', lw=0.25)
    ax[1].set_title('NET_clp')
    ax[1].set_xlim(0, 1)
    ax[1].grid(axis='y')
    ax[1].fill_betweenx(depth, net_clp, color='orange', alpha=0.33)
    twin1 = ax[1].twiny()
    twin1.plot(grn, depth, color='green')
    twin1.set_xlim(0, 150)
    twin1.set_xticks([])
    # ax[2].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[2].plot(net_clp2, depth, color='orange', lw=0.25)
    ax[2].set_title('NET_clp2')
    ax[2].set_xlim(0, 1)
    ax[2].grid(axis='y')
    ax[2].fill_betweenx(depth,net_clp2, color='orange', alpha=0.33)
    twin2 = ax[2].twiny()
    twin2.plot(grn, depth, color='green')
    twin2.set_xlim(0, 150)
    twin2.set_xticks([])
    fig.suptitle(well_sel.well.unique()[0], fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + well + '.png')
    else:
        pass
    return fig.show()
# for well in netvsh_clp2_bal8.well.unique():
#     well_display_net(netvsh_clp2_bal8, well, 'Balakhany VIII', 'NET_clp','NET_clp2', 10, 'print', 'C:\\jupyter\\SPP\plots\\netvsh_clp2_bal8\\')

### netvsh_clp2 bal10

In [114]:
def netvsh_calc_by_gr10(df_interp):
    df_interp.loc[(df_interp.well == 'A01ST1')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                        & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A02')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 50), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A06X')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A09')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 70), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A12')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 70), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A12V')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 70), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A12W')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A12X')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 50), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B05ST1')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B20')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B28')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'C11')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 55), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'C25Z')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'C31')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 50), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'D01Y')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'D26')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 70), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'E08')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'E19')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'E23')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1 
    df_interp.loc[(df_interp.well == 'F02')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'J03')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'J15')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 50), 'NET'] = 1
    return df_interp
df_interp = netvsh_calc_by_gr10(df_interp)    

def vsh_calc_by_gr10(df_interp): 
    gr10_well_list = ['A01ST1', 'A02', 'A02W', 'A02X', 'A02Y', 'A03', 'A06', 'A08X',
                    'A08Y', 'A10Z', 'A11', 'A13', 'A14U', 'A14Z', 'A15', 'A15ST1',
                    'A16ST1', 'A16ST3', 'A16W', 'A19Y', 'AZERI1', 'B04Z', 'B05ST1',
                    'B10Y', 'B10Z', 'B12Z', 'B16W', 'B17', 'B18', 'B18X', 'B22Z',
                    'B25', 'B27', 'B28', 'B33', 'C03', 'C07Z', 'C08', 'C11Z', 'C13',
                    'C15Z', 'C19', 'C22', 'C23', 'C24', 'C25Z', 'C29', 'C30', 'C39',
                    'D05Y', 'D11Y', 'D14Z', 'D16', 'D17Z', 'D18', 'D19', 'D20X',
                    'D20Z', 'D22', 'D23Y', 'D23Z', 'D28', 'E03', 'E04Z', 'E09Y',
                    'E10X', 'E11Y', 'E12', 'E13', 'E15Z', 'E17', 'E18', 'E20', 'E20Z',
                    'E22', 'E38Z', 'F03', 'F04', 'F04Z', 'F05', 'F06', 'G02Z', 'G03',
                    'G04', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07Z', 'H08Z', 'J02Y',
                    'J02Z', 'J07', 'J11', 'J13Y', 'J13Z', 'J14', 'J21']
    df_interp.loc[(df_interp.well.isin(gr10_well_list))  & (df_interp.FORMATION_up == 'Balakhany X') 
                                                         & (df_interp.VSH <= 0.5), 'NET'] = 1
    return df_interp
df_interp = vsh_calc_by_gr10(df_interp)

def well_gr_corr_display(dataset, formation, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']*100
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', marker=dict(color='green')))
    fig.update_xaxes(range=[0, 150])
    fig.update_yaxes(autorange="reversed")
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  marker=dict(color='rgba(245, 190, 20, 0.75)'), fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'))
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=700, width=250, title_text=well, showlegend=False,
                    margin=dict(l=10,r=10,b=10,t=40))
    fig.show()
# well_gr_corr_display(df_interp, 'Balakhany X', 'J11')

In [ ]:
def net_flag_cleaning(dataset_main, fm):
    df_net_bal = dataset_main

    print('Calculation dataframe with h_tst from MD to TST')
    def ntd_calculation_brief(dataset,well, desired_fm, net_var):
        data = dataset[(dataset.well==well)]
        data['TST'].iloc[0] = 0
        data['TST'].iloc[-1] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_list = []
    for well in tqdm(df_net_bal.well.unique()):
        df = ntd_calculation_brief(df_net_bal, well, fm, 'NET')
        df_list.append(df)
    ntd_net = pd.concat(df_list)
    # ntd_net

    # Cleaning NET variable and making up NET_clp with clipped data, join NET_clp to main dataframe
    print('Cleaning NET variable and making up NET_clp with clipped data')
    def ntd_htst_cleaning(dataset_main, dataset_ntd, cutoff):
        ntd_well = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_cutoff = ntd_well[ntd_well.h_tst >= cutoff]
        well_short = dataset_main[['well', 'FORMATION', 'FORMATION_up', 'MD', 'TST', 'GR_N', 'NET', ]]
        net_well = well_short[(well_short.well==well)]
        net_well['NET_clp'] = 0
        for j in range(len(ntd_well_cutoff.well)):
            ntd_top = ntd_well_cutoff.iloc[j, 2].round(3)
            ntd_bot = ntd_well_cutoff.iloc[j, 3].round(3)
            for i in range(len(net_well.TST)):
                well_tst = net_well['TST'].iloc[i].round(3)
                if well_tst >= ntd_top and well_tst <= ntd_bot:
                    net_well['NET_clp'].iloc[i] = 1
        return net_well
    df_list_ntd = []
    for well in tqdm(ntd_net.well.unique()):
        net_well = ntd_htst_cleaning(df_net_bal, ntd_net, 1)
        df_list_ntd.append(net_well)
    net_clp = pd.concat(df_list_ntd)
    # net_clp

    # Cleaning NET_clp from 1-point zero
    print('Cleaning NET_clp from 1-point zero')
    for i in tqdm(range(len(net_clp.NET_clp)-1)):
        if (net_clp.NET_clp.iloc[i] == 0 and  
            net_clp.NET_clp.iloc[i-1] == 1 and 
            net_clp.NET_clp.iloc[i+1] == 1):
            net_clp.NET_clp.iloc[i] = 1        
    df_zero_bal = net_clp[['well', 'MD', 'TST', 'NET_clp', 'FORMATION_up']]
    # df_zero_bal

    print('NET-zero layers searching')
    def ntd_calculation_zero(dataset,well,formation, net_var):
        data = dataset[(dataset.well==well)]
        data.iloc[0, 3] = 1
        data.iloc[-1, 3] = 1
        tst_zero_top = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:]
                    if (data.iloc[i][net_var] == 0 and data.iloc[i-1][net_var] == 1)]
        tst_zero_bot = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:] 
                    if (data.iloc[i][net_var] == 0 and data.iloc[i+1][net_var] == 1)]
        tops_zero = zip(tst_zero_top, tst_zero_bot)
        df_zero_htst = pd.DataFrame(tops_zero, columns=['tst_zero_top', 'tst_zero_bot'])
        df_zero_htst['FORMATION_up'] = formation
        df_zero_htst['well'] = well
        df_zero_htst['h_tst_zero'] = df_zero_htst.tst_zero_bot - df_zero_htst.tst_zero_top
        df_zero_htst = df_zero_htst[['well','FORMATION_up','tst_zero_top','tst_zero_bot','h_tst_zero']]
        return df_zero_htst
    df_zero_list = []
    for well in tqdm(df_zero_bal.well.unique()):
        df = ntd_calculation_zero(df_zero_bal, well, fm, 'NET_clp')
        df_zero_list.append(df)
    ntd_zero = pd.concat(df_zero_list)
    # ntd_zero

    print('NET-zero layers removing')
    def ntd_htst_zero_cleaning(dataset_main, dataset_ntd, cutoff, net_var1, net_var2):
        ntd_well_zero = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_zero_sel = ntd_well_zero[ntd_well_zero.h_tst_zero <= cutoff]
        well_zero_short = dataset_main[['well','FORMATION_up','MD','TST', net_var1, 'GR_N', 'NET', 'FORMATION']]
        well_zero_short[net_var2] = well_zero_short[net_var1]
        well_zero_sel = well_zero_short[(well_zero_short.well==well)]
        for j in range(len(ntd_well_zero_sel.well)):
            ntd_zero_top = ntd_well_zero_sel.iloc[j, 2].round(3)
            ntd_zero_bot = ntd_well_zero_sel.iloc[j, 3].round(3)
            for i in range(len(well_zero_sel.TST)):
                well_zero_tst = well_zero_sel['TST'].iloc[i].round(3)
                if well_zero_tst >= ntd_zero_top and well_zero_tst <= ntd_zero_bot:
                    well_zero_sel[net_var2].iloc[i] = 1
        return well_zero_sel
    df_list_ntd_zero = []
    for well in tqdm(ntd_zero.well.unique()):
        well_zero_sel = ntd_htst_zero_cleaning(net_clp, ntd_zero, 0.75, 'NET_clp', 'NET_clp2')
        df_list_ntd_zero.append(well_zero_sel)
    net_clp2 = pd.concat(df_list_ntd_zero)
    return net_clp2
black_list_wells10 = ['AZERI4', 'AZERI5', 'CHIRAG1', 'CHIRAG4', 'CHIRAG5', 'CHIRAG6', 'E05Z', 'E13Y', 'E21A',
                    'GCA1','GCA2', 'GCA4', 'GCA4Z', 'GCA5', 'GCA5Z', 'GCA6', 'GCA6Y', 'GCA6Z','GCA7']
df_bal10_vsh = df_interp[(df_interp.well.isin(vsh_clean_bal10.well.unique()))
                            & (~df_interp.well.isin(black_list_wells10)) 
                            & (df_interp.FORMATION_up == 'Balakhany X')]
netvsh_clp2_bal10 = net_flag_cleaning(df_bal10_vsh, 'Balakhany X')

def vsh_bal10_net_corr(test_net_clp2):
    test_net_clp2 = netvsh_clp2_bal10.copy()
    test_net_clp2.loc[(test_net_clp2.well == 'A03Y')  & (test_net_clp2.TST >= 1274.5) 
                                                    & (test_net_clp2.TST <= 1279), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A06X')  & (test_net_clp2.TST >= 485.5) 
                                                    & (test_net_clp2.TST <= 490), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A06X')  & (test_net_clp2.TST >= 521.5) 
                                                    & (test_net_clp2.TST <= 526.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A06X')  & (test_net_clp2.TST >= 531) 
                                                    & (test_net_clp2.TST <= 537), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A07')  & (test_net_clp2.TST >= 638) 
                                                    & (test_net_clp2.TST <= 642), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A07')  & (test_net_clp2.TST >= 645) 
                                                    & (test_net_clp2.TST <= 649), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A08')  & (test_net_clp2.TST >= 571.8) 
                                                    & (test_net_clp2.TST <= 573.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A08')  & (test_net_clp2.TST >= 602) 
                                                    & (test_net_clp2.TST <= 607.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A08')  & (test_net_clp2.TST >= 624) 
                                                    & (test_net_clp2.TST <= 631.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A08Z')  & (test_net_clp2.TST >= 962.6) 
                                                    & (test_net_clp2.TST <= 966.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12X')  & (test_net_clp2.TST >= 170) 
                                                    & (test_net_clp2.TST <= 175.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12X')  & (test_net_clp2.TST >= 182.5) 
                                                    & (test_net_clp2.TST <= 187.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12YST1')  & (test_net_clp2.TST >= 1018.4) 
                                                    & (test_net_clp2.TST <= 1030.4), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12YST1')  & (test_net_clp2.TST >= 1043.5) 
                                                    & (test_net_clp2.TST <= 1053.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12YST1')  & (test_net_clp2.TST >= 1057.1) 
                                                    & (test_net_clp2.TST <= 1059.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A20')  & (test_net_clp2.TST >= 550) 
                                                    & (test_net_clp2.TST <= 565), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'A20')  & (test_net_clp2.TST >= 677.2) 
                                                    & (test_net_clp2.TST <= 683.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A20')  & (test_net_clp2.TST >= 625) 
                                                    & (test_net_clp2.TST <= 626), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'A20')  & (test_net_clp2.TST >= 642.8) 
                                                    & (test_net_clp2.TST <= 644), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'A20Y')  & (test_net_clp2.TST >= 55.6) 
                                                    & (test_net_clp2.TST <= 58.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 2161.6) 
                                                    & (test_net_clp2.TST <= 2163.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 2167) 
                                                    & (test_net_clp2.TST <= 2174), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B11')  & (test_net_clp2.TST >= 2211) 
                                                    & (test_net_clp2.TST <= 2220), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B11')  & (test_net_clp2.TST >= 2230) 
                                                    & (test_net_clp2.TST <= 2241.6), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B28')  & (test_net_clp2.TST >= 2295.5) 
                                                & (test_net_clp2.TST <= 2301.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B32')  & (test_net_clp2.TST >= 2540) 
                                                    & (test_net_clp2.TST <= 2550), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B34')  & (test_net_clp2.TST >= 2477) 
                                                    & (test_net_clp2.TST <= 2483), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B34')  & (test_net_clp2.TST >= 2548) 
                                                    & (test_net_clp2.TST <= 2557), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 2163.7) 
                                                    & (test_net_clp2.TST <= 2166.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C03Z')  & (test_net_clp2.TST >= 993) 
                                                    & (test_net_clp2.TST <= 1001), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C03Z')  & (test_net_clp2.TST >= 1002) 
                                                    & (test_net_clp2.TST <= 1008.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C03Z')  & (test_net_clp2.TST >= 1060.4) 
                                                    & (test_net_clp2.TST <= 1064), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2327.4) 
                                                    & (test_net_clp2.TST <= 2330.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2356) 
                                                    & (test_net_clp2.TST <= 2360), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2394) 
                                                    & (test_net_clp2.TST <= 2399), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2400.5) 
                                                    & (test_net_clp2.TST <= 2402), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C25Z')  & (test_net_clp2.TST >= 624.5) 
                                                    & (test_net_clp2.TST <= 632.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C25Z')  & (test_net_clp2.TST >= 696.5) 
                                                    & (test_net_clp2.TST <= 701.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 2235.5) 
                                                    & (test_net_clp2.TST <= 2249), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C32')  & (test_net_clp2.TST >= 2082) 
                                                    & (test_net_clp2.TST <= 2086), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D02Z')  & (test_net_clp2.TST >= 440) 
                                                    & (test_net_clp2.TST <= 495), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D02Z')  & (test_net_clp2.TST >= 504.8) 
                                                    & (test_net_clp2.TST <= 506.6), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D02Z')  & (test_net_clp2.TST >= 523) 
                                                    & (test_net_clp2.TST <= 536), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D03')  & (test_net_clp2.TST >= 3203) 
                                                    & (test_net_clp2.TST <= 3223), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D04')  & (test_net_clp2.TST >= 3161) 
                                                    & (test_net_clp2.TST <= 3210), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D04')  & (test_net_clp2.TST >= 3259) 
                                                    & (test_net_clp2.TST <= 3281), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D06')  & (test_net_clp2.TST >= 3105) 
                                                    & (test_net_clp2.TST <= 3147), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D06')  & (test_net_clp2.TST >= 3156) 
                                                    & (test_net_clp2.TST <= 3165), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D06')  & (test_net_clp2.TST >= 3236) 
                                                    & (test_net_clp2.TST <= 3245), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D06')  & (test_net_clp2.TST >= 3226) 
                                                    & (test_net_clp2.TST <= 3235), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D09')  & (test_net_clp2.TST >= 3359) 
                                                    & (test_net_clp2.TST <= 3362.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3320) 
                                                    & (test_net_clp2.TST <= 3392), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3425) 
                                                    & (test_net_clp2.TST <= 3447), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D20')  & (test_net_clp2.TST >= 2966) 
                                                    & (test_net_clp2.TST <= 3011), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D20')  & (test_net_clp2.TST >= 3055) 
                                                    & (test_net_clp2.TST <= 3097), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 3080) 
                                                    & (test_net_clp2.TST <= 3127), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 3157) 
                                                    & (test_net_clp2.TST <= 3175), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D26')  & (test_net_clp2.TST >= 2877.1) 
                                                    & (test_net_clp2.TST <= 2880.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E01Z')  & (test_net_clp2.TST >= 66.8) 
                                                    & (test_net_clp2.TST <= 69.3), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E04')  & (test_net_clp2.TST >= 0) 
                                                    & (test_net_clp2.TST <= 5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E04Y')  & (test_net_clp2.TST >= 0) 
                                                    & (test_net_clp2.TST <= 15.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E04Y')  & (test_net_clp2.TST >= -8) 
                                                    & (test_net_clp2.TST <= 0), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E08')  & (test_net_clp2.TST >= 31) 
                                                    & (test_net_clp2.TST <= 35.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E10W')  & (test_net_clp2.TST >= 2800) 
                                                    & (test_net_clp2.TST <= 2810), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E10W')  & (test_net_clp2.TST >= 2839) 
                                                    & (test_net_clp2.TST <= 2841), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E10W')  & (test_net_clp2.TST >= 2843.3) 
                                                    & (test_net_clp2.TST <= 2844.4), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E10W')  & (test_net_clp2.TST >= 2848.2) 
                                                    & (test_net_clp2.TST <= 2849), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E11Z')  & (test_net_clp2.TST >= 24.8) 
                                                    & (test_net_clp2.TST <= 28.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E16X')  & (test_net_clp2.TST >= 713) 
                                                    & (test_net_clp2.TST <= 724), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E16X')  & (test_net_clp2.TST >= 740) 
                                                    & (test_net_clp2.TST <= 747), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E16X')  & (test_net_clp2.TST >= 749.5) 
                                                    & (test_net_clp2.TST <= 756.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E19')  & (test_net_clp2.TST >= 2140) 
                                                    & (test_net_clp2.TST <= 2150), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'F01Z')  & (test_net_clp2.TST >= -118) 
                                                    & (test_net_clp2.TST <= -94), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'F01Z')  & (test_net_clp2.TST >= -39) 
                                                    & (test_net_clp2.TST <= -36), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'F01Z')  & (test_net_clp2.TST >= -35.7) 
                                                    & (test_net_clp2.TST <= -32.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'G02')  & (test_net_clp2.TST >= 2530) 
                                                    & (test_net_clp2.TST <= 2540), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'G05')  & (test_net_clp2.TST >= 1933.3) 
                                                    & (test_net_clp2.TST <= 1940.6), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'H01Z')  & (test_net_clp2.TST >= 445) 
                                                    & (test_net_clp2.TST <= 446), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'H01Z')  & (test_net_clp2.TST >= 458.5) 
                                                    & (test_net_clp2.TST <= 461.1), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'J03')  & (test_net_clp2.TST >= 2430) 
                                                    & (test_net_clp2.TST <= 2432), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'J13')  & (test_net_clp2.TST >= 2295) 
                                                    & (test_net_clp2.TST <= 2298), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'J13')  & (test_net_clp2.TST >= 2334) 
                                                    & (test_net_clp2.TST <= 2337), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J15')  & (test_net_clp2.TST >= 2491) 
                                                    & (test_net_clp2.TST <= 2496), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J15')  & (test_net_clp2.TST >= 2531.5) 
                                                    & (test_net_clp2.TST <= 2533.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J18A')  & (test_net_clp2.TST >= 1825.5) 
                                                    & (test_net_clp2.TST <= 1830.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J30')  & (test_net_clp2.TST >= 1759) 
                                                    & (test_net_clp2.TST <= 1763), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'J30')  & (test_net_clp2.TST >= 1775) 
                                                    & (test_net_clp2.TST <= 1780), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J32')  & (test_net_clp2.TST >= 1803) 
                                                    & (test_net_clp2.TST <= 1805), 'NET_clp2'] = 0
    return test_net_clp2
netvsh_clp2_bal10 = vsh_bal10_net_corr(netvsh_clp2_bal10)

df_bal10_vshclp2 = df_bal10_vsh.set_index(['well','TST']).join(
                   netvsh_clp2_bal10.drop(['FORMATION_up','MD','GR_N','NET','FORMATION'], axis=1).set_index(['well','TST'])).reset_index()
df_bal10_vshclp2.loc[df_bal10_vshclp2.well.isin(phit_clean_bal10.well.unique()), 'phit_flag'] = 1
df_bal10_vshclp2['phit_flag'] = df_bal10_vshclp2['phit_flag'].fillna(0)

def perm_ds_us_calc(dataset):
    def perm_ds_1(x):
        return 10**(39139.46 * pow(x, 5) - 11140.04 * pow(x, 4) + 855.2176 * pow(x, 3) + 7.04505 * pow(x, 2) + 5.750233 * x - 1.997085)
    def perm_us_1(x):
        return 10**(1638286 * pow(x, 5) - 1396883 * pow(x, 4) + 468324.8 * pow(x, 3) - 76974.79 * pow(x, 2) + 6217.262 * x - 198.5042)
    def perm_ds_2(x):
        return 10**(+ 5675.143 * pow(x, 5) - 11106.91 * pow(x, 4) + 8608.366 * pow(x, 3) - 3318.893 * pow(x, 2) + 644.7713 * x - 48.16968)
    def perm_us_2(x):
        return 10**(+ 81.59968 * pow(x, 5) - 275.5442 * pow(x, 4) + 364.9522 * pow(x, 3) - 238.8838 * pow(x, 2) + 79.97139 * x - 7.15)
    df = dataset
    try:
        df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = df.PHIT.apply(perm_ds_1)
        df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = df.PHIT.apply(perm_us_1)
        df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_DS'] = df.PHIT.apply(perm_ds_2)
        df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_US'] = df.PHIT.apply(perm_us_2)
    except:
        df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = 0
        df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = 0
        df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_DS'] = 0
        df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_US'] = 0
    df['PERM_DS'] = df['PERM_DS'].fillna(0)
    df['PERM_US'] = df['PERM_US'].fillna(0)
    return df
df_bal10_vshclp2 = perm_ds_us_calc(df_bal10_vshclp2)

In [ ]:
df_bal10_vshclp2.loc[df_bal10_vshclp2.well.isin(['B27', 'B28', 'B44']), 'field'] = 'CENTRAL AZERI'
df_bal10_vshclp2.loc[df_bal10_vshclp2.well.isin(['C27Y','C19Z', 'C27X']), 'field'] = 'WEST AZERI'
df_bal10_vshclp2.loc[df_bal10_vshclp2.well.isin(['D09Y','D19Z','D19Y']), 'field'] = 'EAST AZERI'
df_bal10_vshclp2.loc[df_bal10_vshclp2.well.isin(['E12Z', 'E20Y']), 'field'] = 'DWG'
df_bal10_vshclp2.loc[df_bal10_vshclp2.well.isin(['J09Z', 'J13X']), 'field'] = 'WEST CHIRAG'

df_bal10_vshclp2_v3 = khtst_calculation(df_bal10_vshclp2)
df_bal10_vshclp2_v3 = df_bal10_vshclp2_v3[~df_bal10_vshclp2_v3.well.isin(['A19Z'])]

df_bal10_vshclp2_v3 = df_bal10_vshclp2_v3[df_bal10_vshclp2_v3.FORMATION != 'Balakhany X']

tst_interv = df_bal10_vshclp2_v3.groupby('well')['TST'].apply(lambda x: x.iloc[-1] - x.iloc[0])
df_bal10_vshclp2_v3 = df_bal10_vshclp2_v3.merge(tst_interv, on='well', suffixes=('', '_interv'))

tst_interv_fu = df_bal10_vshclp2_v3.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[-1] - x.iloc[0])
df_bal10_vshclp2_v3 = df_bal10_vshclp2_v3.merge(tst_interv_fu, on=['well','FORMATION'], suffixes=('', '_interv_fu'))

fluid_code = pd.read_csv(r'C:\jupyter\SPP\input\fluidcode_sql_tl.csv', names = ['well', 'MD','fluid_code'])
fluid_code = fluid_code.astype({'well':'string'})
fluid_code['fluid_code'] = fluid_code['fluid_code'].fillna(0)
fluid_code = fluid_code.drop_duplicates(subset=['well', 'MD'])
df_bal10_vshclp2_v3_fluid = df_bal10_vshclp2_v3.set_index(['well','MD']).join(fluid_code.set_index(['well','MD'])).reset_index().drop(columns=['V'], axis=1)

calcite10 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_calc.csv').drop(columns=['Unnamed: 0'])
calcite10 = calcite10[['well','TST','calc', 'calc_net']]
df_bal10_vshclp2_v4 = df_bal10_vshclp2_v3_fluid.set_index(['well','TST']).join(calcite10.set_index(['well','TST'])).reset_index()

df_bal10_vshclp2_v4['phitd_npss'] = (0.6 - df_bal10_vshclp2_v4['NPSS'])*100 - (df_bal10_vshclp2_v4['RHOB'] - 1.67)*60

def interpolate_by_depth_fm_run_v2(df, step):
    df_tst = df[df.TST.notna()].round({'MD':1})
    
    def interpolate_by_depth_fm_v2(one_well, step):
        one_well = one_well.sort_values(by='TST')
        well_name = one_well["well"].iloc[0]
        formation = one_well["FORMATION"].iloc[0]
        formation_up = one_well["FORMATION_up"].iloc[0]
        field = one_well["field"].iloc[0]
        data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
        starting_tst = one_well["TST"].iloc[0]
        new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
        col_lst = []
        for col in one_well.columns:
            if col not in ['well','FORMATION_up', 'FORMATION','field']:
                interp = interp1d(one_well['TST'], one_well[col], kind='linear', fill_value="extrapolate")
                new_data = {col: interp(new_TST_values)}
                new_df = pd.DataFrame(new_data)
                col_lst.append(new_df)
        new_df = pd.concat(col_lst, axis=1)
        new_df['well'] = well_name
        new_df['TST'] = new_TST_values
        new_df['FORMATION'] = formation
        new_df['FORMATION_up'] = formation_up
        new_df['field'] = field #['VSH_smooth', 'NET_smooth', 'NET_orig', 'NET_VSH_orig'] not in index"
        new_df = new_df[[   'well', 
                            'TST', 'tst_index', 'MD',  'DEVI', 'HAZI', 'NET', 'NET_VSH', 'LPERM',
                            'PHIT', 'GR_N', 'VSH', 'NPSS', 'RHOB', 'RDEEP', 'SON', 'SONSH',
                            'TVD_SCS', 'X_traj', 'Y_traj', 'Xmean', 'Ymean', 'RHOF', 'RHOMA',
                            'tst_sample', 'NET_clp', 'NET_clp2', 'phit_flag', 'PERM_DS', 'PERM_US',
                            'k_htst', 'KHtst', 'TST_interv', 'TST_interv_fu', 'fluid_code', 'calc',
                            'calc_net', 'phitd_npss']]
        return new_df
    df_lst = []
    for well in tqdm(df_tst.well.unique()):
        well_data = df_tst[df_tst.well == well]
        well_data_interp = interpolate_by_depth_fm_v2(well_data, 0.1)
        df_lst.append(well_data_interp)
    df_interp = pd.concat(df_lst)
    df_interp = df_interp.round({'MD':1, 'TVD_SCS':1, 'TST':1})
    
    print('Start joining')
    def well_bal_interp_join(dataset):
        df_tst = df[(df.TST.notna()) & (df.FORMATION_up.notna())].round({'MD':1})
        data_fu = df_tst[['well','MD','FORMATION_up', 'FORMATION', 'field']]
        well_join = dataset.set_index(['well','MD']).join(data_fu.set_index(['well','MD'])).reset_index()
        well_join.insert(3, 'FORMATION_up', well_join.pop('FORMATION_up'))
        well_join.insert(4, 'FORMATION', well_join.pop('FORMATION'))
        well_join.insert(5, 'tst_index', well_join.pop('tst_index'))
        return well_join
    well_interp_v2 = well_bal_interp_join(df_interp)
    well_interp_v2.loc[well_interp_v2.NET_VSH > 0, 'NET_VSH'] = 1
    well_interp_v2.loc[well_interp_v2.NET > 0, 'NET'] = 1
    
    df_lst_2 = []
    for well in well_interp_v2.well.unique():
        field_data = well_interp_v2[well_interp_v2.well == well]
        field_data = field_data.drop_duplicates()
        field_data.field = field_data.field.fillna(method = 'ffill')
        field_data.field = field_data.field.fillna(method = 'bfill')
        field_data.FORMATION_up = field_data.FORMATION_up.fillna(method = 'ffill')
        field_data.FORMATION_up = field_data.FORMATION_up.fillna(method = 'bfill')
        field_data.FORMATION = field_data.FORMATION.fillna(method = 'ffill')
        field_data.FORMATION = field_data.FORMATION.fillna(method = 'bfill')
        df_lst_2.append(field_data)
    well_interp_v3 = pd.concat(df_lst_2)

    return well_interp_v3

df_bal10_vshclp2_v4 = interpolate_by_depth_fm_run_v2(df_bal10_vshclp2_v4, 0.1)

df_bal10_vshclp2_v4.to_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_v4.csv', index=False)

In [ ]:
df_bal10_vshclp2_v4[df_bal10_vshclp2_v4.well == 'E31Z']

In [123]:
# for wellname in df_bal10_vshclp2_v3.well.unique():
#     print(wellname)
#     well_display_logs(df_bal10_vshclp2_v3, wellname, 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal10\\')
# well_display_logs(df_bal10_vshclp2_v4, 'A12W', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal10\\')

In [124]:
def well_gr_phit_corr_display(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    phit = well_sel['PHIT']
    net = well_sel[var]*100
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=2, subplot_titles=('GR', 'PHIT'))
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', 
                             line=dict(color='green')), row=1, col=1)
    
    fig.update_xaxes(range = [0,150])
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 150], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=850, width=400,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50))
    fig.show()
# well_gr_phit_corr_display(df_bal10_vshclp2, 'Balakhany X','NET_clp2', 'E30Y')

In [125]:
def well_display_net(dataset, well, formation, net1, net2, depth_step, print_flag, path):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']
    net_clp = well_sel[net1]
    net_clp2 = well_sel[net2]
    
    fig, ax = plt.subplots(1,3, figsize=(6,8), sharey=True)
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(net, depth, color='orange', lw=0.25)
    ax[0].fill_betweenx(depth,net, color='orange', alpha=0.33)
    ax[0].invert_yaxis()
    ax[0].set_xlim(0, 1) 
    ax[0].set_title('NET')
    well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
        ax[0].text(0.1, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
    twin0 = ax[0].twiny()
    twin0.plot(grn, depth, color='green')
    twin0.set_xlim(0, 150)
    twin0.set_xticks([])
    # ax[1].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[1].plot(net_clp, depth, color='orange', lw=0.25)
    ax[1].set_title('NET_clp')
    ax[1].set_xlim(0, 1)
    ax[1].grid(axis='y')
    ax[1].fill_betweenx(depth, net_clp, color='orange', alpha=0.33)
    twin1 = ax[1].twiny()
    twin1.plot(grn, depth, color='green')
    twin1.set_xlim(0, 150)
    twin1.set_xticks([])
    # ax[2].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[2].plot(net_clp2, depth, color='orange', lw=0.25)
    ax[2].set_title('NET_clp2')
    ax[2].set_xlim(0, 1)
    ax[2].grid(axis='y')
    ax[2].fill_betweenx(depth,net_clp2, color='orange', alpha=0.33)
    twin2 = ax[2].twiny()
    twin2.plot(grn, depth, color='green')
    twin2.set_xlim(0, 150)
    twin2.set_xticks([])
    fig.suptitle(well_sel.well.unique()[0], fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + well + '.png')
    else:
        pass
    return fig.show()
# for well in netvsh_clp2_bal10.well.unique():
#     well_display_net(netvsh_clp2_bal10, well, 'Balakhany X', 'NET_clp','NET_clp2', 10, 'print', 'C:\\jupyter\\SPP\plots\\netvsh_clp2_bal10\\')

## NET data uploading

In [188]:
df_bal8_v4 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v4.csv')
well_phit_flag8 = df_bal8_v4[df_bal8_v4.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
df_bal8_v4_flag = df_bal8_v4[df_bal8_v4.well.isin(well_phit_flag8)]

df_bal10_v4 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_v4.csv')
well_phit_flag10 = df_bal10_v4[df_bal10_v4.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
df_bal10_v4_flag = df_bal10_v4[df_bal10_v4.well.isin(well_phit_flag10)]

In [127]:
# well_display_logs(df_bal8_azr_v3, 'C01', 'Balakhany VIII', 'NET_VSH', 'NET', 'TVD_SCS', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')

In [128]:
def well_display_logs_cube(dataset, dataset_cube, wellname, fmname, net_var, comments, ref_depth, depth_step, print_flag, path):
    """
    dataset = df_bal or something else
    net_var = NET or FLUIDS_int
    comments = put what you want
    ref_depth = MD or TST
    depth_step = step for ticks on the diagramm
    print = 1 if we want to print the plot
    well_display_logs(df_bal8_azr_vshclp2_flag, 'C01', 'Balakhany VIII', 'NET_clp2', 'df_interp', 'TST')
    """
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    data_cube = dataset_cube[(dataset_cube.well==wellname) & (dataset_cube.FORMATION_up == fmname)]
    depth = data[ref_depth]
    depth_cube = data_cube[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    vsh_cube = data_cube['VSH_cube']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    phit_cube = data_cube['PHIT_cube']
    net_init = data['NET'] 
    net = data[net_var]
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    fig, ax = plt.subplots(1,4, figsize=(7,7), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.plot(vsh_cube, depth_cube, color='#141313', alpha=0.5, zorder=5)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(axis='x')
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(axis='x')
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)

    twin1.plot(net_init, depth, color='orange', linewidth=0.1)
    twin1.fill_betweenx(depth,net_init, color='orange', alpha=0.25)

    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].plot(phit_cube, depth_cube, color='#e91717', linestyle='dashed', alpha=1)
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].grid(axis='x')
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    # twin2.grid(axis='x')
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(comments), fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png')
    else:
        pass
    plt.show()
# for wellname in df_bal8_azr_vshclp2.well.unique():
#     net2 = (df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well == wellname]['NET_clp2'].sum()*0.1).round(1)
#     tst_int = int(df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well == wellname]['TST'].iloc[-1] - df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well == wellname]['TST'].iloc[0])
#     well_display_logs_cube(df_bal8_azr_vshclp2, df_cube, wellname, 'Balakhany VIII', 'NET_clp2', f'tst: {tst_int}m net2: {net2}m', 'TST', 10, 
#                            'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal8_cube\\')

# for wellname in df_bal10_vshclp2.well.unique():
#     net2 = (df_bal10_vshclp2[df_bal10_vshclp2.well == wellname]['NET_clp2'].sum()*0.1).round(1)
#     tst_int = int(df_bal10_vshclp2[df_bal10_vshclp2.well == wellname]['TST'].iloc[-1] - df_bal10_vshclp2[df_bal10_vshclp2.well == wellname]['TST'].iloc[0])
#     well_display_logs_cube(df_bal10_vshclp2, df_cube, wellname, 'Balakhany X', 'NET_clp2', f'tst: {tst_int}m net2: {net2}m', 'TST', 10, 
                        #    'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal10_cube\\')

In [129]:
def ntd_dataframe_calculation(dataset, fm, net_var):
    print('Calculation NTD based on NET_clp2')
    def ntd_calculation(dataset, well, fm, net_var):
        data = dataset[(dataset.well==well)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_recalc_list = []
    for well in tqdm(dataset.well.unique()):
        df = ntd_calculation(dataset, well, fm, net_var)
        df_recalc_list.append(df)
    ntd_net2 = pd.concat(df_recalc_list)
    ntd_net2.drop_duplicates(inplace=True)

    print('Calculation values for NTD')
    def ntd_properties_averaging(dataset_ntd, dataset_logs, fm):
        well_data = []
        well_formation = fm
        df_lst = []
        for well in tqdm(dataset_ntd.well.unique()):
            ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
            well_avgprop_sel = dataset_logs[(dataset_logs.well==well)]
            fm_top = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[0]
            fm_bot = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[-1]
            well_phit = []
            well_vsh = []
            well_perm = []
            well_top = []
            well_bot = []
            well_h = []
            well_layer_top = []
            well_layer_bot = []
            well_fm_top = []
            well_fm_bot = []
            well_name = []
            well_fm = []
            well_kh = []
            for layers in range(len(ntd_well_avgprop.well)):
                ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
                ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
                ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
                phit_lst = []
                vsh_lst = []
                perm_lst = []
                depth_tst = []
                khtst_lst = []
                for depth in range(len(well_avgprop_sel.TST)):
                    well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                    if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                        phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                        vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                        perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
                        depth_tst.append(well_avgprop_sel['TST'].iloc[depth])
                        khtst_lst.append(well_avgprop_sel['KHtst'].iloc[depth])
                well_name.append(well)
                well_fm.append(well_formation)
                well_phit.append(mean(phit_lst))
                well_vsh.append(mean(vsh_lst))
                well_perm.append(mean(perm_lst))
                well_layer_top.append(depth_tst[0])
                well_layer_bot.append(depth_tst[-1])
                well_kh.append((khtst_lst[0] - khtst_lst[-1]))
                well_h.append(ntd_h)
                well_top.append(ntd_top)
                well_bot.append(ntd_bot)
                well_fm_top.append(fm_top)
                well_fm_bot.append(fm_bot)
                well_data = zip(well_name,well_fm,well_phit, well_vsh, well_perm, well_kh, 
                                well_h, well_top, well_bot, well_layer_top, well_layer_bot,
                                well_fm_top, well_fm_bot)
                well_df = pd.DataFrame(well_data, columns=[ 'well','FORMATION_up',        
                                                            'phit_avg',
                                                            'vsh_avg', 
                                                            'perm_avg',
                                                            'khtst',
                                                            'htst',
                                                            'top_tst',
                                                            'bot_tst',
                                                            'layer_top_tst',
                                                            'layer_bot_tst',
                                                            'fm_top_tst',
                                                            'fm_bot_tst'])
                well_df['not_htst'] = well_df['top_tst'].shift(-1)-well_df['bot_tst']
                well_df = well_df[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'khtst',
                                   'htst', 'not_htst','top_tst', 'bot_tst',
                                   'layer_top_tst', 'layer_bot_tst', 
                                   'fm_top_tst', 'fm_bot_tst']]
            df_lst.append(well_df)
        result = pd.concat(df_lst)
        return result
    ntd_val = ntd_properties_averaging(ntd_net2, dataset, fm)

    def nothtst_nan_filling(dataset_ntd, fm):
        def nan_change_diff_fmbottom(dataset, wellname, fmname):
            row_change = dataset[(dataset.well == wellname) & (dataset.FORMATION_up == fmname) & (dataset.not_htst.isna())]
            row_change['not_htst'] = row_change['fm_bot_tst'] - row_change['bot_tst']
            return row_change
        df_list = []
        for wellname in dataset_ntd.well.unique():
            df = nan_change_diff_fmbottom(dataset_ntd, wellname, fm)
            df_list.append(df)
        res_df_list = pd.concat(df_list)
        result = pd.concat([dataset_ntd, res_df_list])
        result = result.sort_values(by=['well','top_tst'])
        result_final = result.dropna(subset=['not_htst'], axis=0)

        return result_final
    ntd_val_clean = nothtst_nan_filling(ntd_val, fm)

    def top_phit_bot_clustering(dataset):
        print('Top & bot calculation')
        def top_phit_bot_collection_run(dataset):
            def top_phit_bot_collection(dataset, wellname):
                data = dataset[dataset.well == wellname]
                data['top_htst'] = data['top_tst'] - data['fm_top_tst']
                data['top_htst'].iloc[1:] = data['not_htst'].iloc[:-1]
                data['bot_htst'] = data['not_htst']
                data = data[[   'well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg','khtst',
                                'top_htst','htst','bot_htst', 'layer_top_tst', 'layer_bot_tst', 
                                'fm_top_tst', 'fm_bot_tst']]
                return data
            df_lst = []
            for wellname in tqdm(dataset.well.unique()):
                res_df = top_phit_bot_collection(dataset, wellname)
                df_lst.append(res_df)
            top_phi_bot_cluster = pd.concat(df_lst).reset_index(drop=True)
            return top_phi_bot_cluster
        top_phi_bot_cluster = top_phit_bot_collection_run(dataset)

        def top_phit_bot_ntg_run(dataset):
            def top_phit_bot_ntg(dataset, wellname):
                ntg = []
                data = dataset[dataset.well == wellname].reset_index(drop=True)
                for ind, row in data.iterrows():
                    if ind == 0:
                        ntg.append(row['htst']/(row['bot_htst'] + row['htst']))
                    if ind != 0:
                        ntg.append(row['htst']/(row['bot_htst'] + row['htst'] + row['top_htst']))
                    if ind == len(data):
                        ntg.append(row['htst']/(row['top_htst'] + row['htst']))
                result = pd.concat([data, pd.DataFrame({'ntg':ntg})], axis=1)
                return result
            df_lst = []
            for wellname in dataset.well.unique():
                df = top_phit_bot_ntg(dataset, wellname)
                df_lst.append(df)
            top_phi_bot_cluster_ntg = pd.concat(df_lst).reset_index(drop=True)
            return top_phi_bot_cluster_ntg
        top_phi_bot_cluster_ntg = top_phit_bot_ntg_run(top_phi_bot_cluster)
        
        return top_phi_bot_cluster_ntg
    ntd_top_phi_bot = top_phit_bot_clustering(ntd_val_clean)

    return ntd_top_phi_bot.reset_index(drop=True)
# ntd_top_vsh_bot8 = ntd_dataframe_calculation(df_bal8_azr_v3, 'Balakhany VIII', 'NET_clp2')
# ntd_top_phi_bot8 = ntd_dataframe_calculation(df_bal8_azr_v3_flag, 'Balakhany VIII', 'NET_clp2')
# ntd_top_vsh_bot10 = ntd_dataframe_calculation(df_bal10_vshclp2_v3, 'Balakhany X', 'NET_clp2')
# ntd_top_phi_bot10 = ntd_dataframe_calculation(df_bal10_vshclp2_v3_flag, 'Balakhany X', 'NET_clp2')

In [ ]:
def ntd_dataframe_calculation_bp(dataset, fm, net_var):
    print(f'Calculation NTD based on {net_var}')
    def ntd_calculation(dataset, well, fm, net_var):
        data = dataset[(dataset.well==well)]
        idx = []
        for i in range(len(data.columns)):
            if data.columns[i] == net_var:
                idx.append(i)
        data.iloc[0, idx[0]] = 0
        data.iloc[-1, idx[0]] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_recalc_list = []
    for well in tqdm(dataset.well.unique()):
        df = ntd_calculation(dataset, well, fm, net_var)
        df_recalc_list.append(df)
    ntd_net2 = pd.concat(df_recalc_list)
    ntd_net2.drop_duplicates(inplace=True)
    ntd_net2 = ntd_net2[ntd_net2.h_tst > 0]

    print('Calculation values for NTD')
    def ntd_properties_averaging(dataset_ntd, dataset_logs, fm):
        well_data = []
        well_formation = fm
        df_lst = []
        for well in tqdm(dataset_ntd.well.unique()):
            ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
            well_avgprop_sel = dataset_logs[(dataset_logs.well==well)]
            fm_top = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[0]
            fm_bot = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[-1]
            well_phit = []
            well_vsh = []
            well_perm = []
            well_top = []
            well_bot = []
            well_h = []
            well_layer_top = []
            well_layer_bot = []
            well_fm_top = []
            well_fm_bot = []
            well_name = []
            well_fm = []
            well_kh = []
            for layers in range(len(ntd_well_avgprop.well)):
                ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
                ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
                ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
                phit_lst = []
                vsh_lst = []
                perm_lst = []
                depth_tst = []
                khtst_lst = []
                for depth in range(len(well_avgprop_sel.TST)):
                    well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(5)
                    if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                        phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                        vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                        perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
                        depth_tst.append(well_avgprop_sel['TST'].iloc[depth])
                        khtst_lst.append(well_avgprop_sel['KHtst'].iloc[depth])
                well_name.append(well)
                well_fm.append(well_formation)
                well_phit.append(mean(phit_lst))
                well_vsh.append(mean(vsh_lst))
                well_perm.append(mean(perm_lst))
                well_layer_top.append(depth_tst[0])
                well_layer_bot.append(depth_tst[-1])
                well_kh.append((khtst_lst[0] - khtst_lst[-1]))
                well_h.append(ntd_h)
                well_top.append(ntd_top)
                well_bot.append(ntd_bot)
                well_fm_top.append(fm_top)
                well_fm_bot.append(fm_bot)
                well_data = zip(well_name,well_fm,well_phit, well_vsh, well_perm, well_kh,
                                well_h, well_top, well_bot, well_layer_top, well_layer_bot,
                                well_fm_top, well_fm_bot)
                well_df = pd.DataFrame(well_data, columns=[ 'well','FORMATION_up',        
                                                            'phit_avg',
                                                            'vsh_avg', 
                                                            'perm_avg',
                                                            'khtst',
                                                            'htst',
                                                            'top_tst',
                                                            'bot_tst',
                                                            'layer_top_tst',
                                                            'layer_bot_tst',
                                                            'fm_top_tst',
                                                            'fm_bot_tst'])
                well_df['not_htst'] = well_df['top_tst'].shift(-1)-well_df['bot_tst']
                well_df = well_df[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'khtst',
                                   'htst', 'not_htst','top_tst', 'bot_tst', 
                                   'layer_top_tst', 'layer_bot_tst',
                                   'fm_top_tst', 'fm_bot_tst']]
            df_lst.append(well_df)
        result = pd.concat(df_lst)
        return result
    ntd_val = ntd_properties_averaging(ntd_net2, dataset, fm)

    def nothtst_nan_filling(dataset_ntd, fm):
        def nan_change_diff_fmbottom(dataset, wellname, fmname):
            row_change = dataset[(dataset.well == wellname) & (dataset.FORMATION_up == fmname) & (dataset.not_htst.isna())]
            row_change['not_htst'] = row_change['fm_bot_tst'] - row_change['bot_tst']
            return row_change
        df_list = []
        for wellname in dataset_ntd.well.unique():
            df = nan_change_diff_fmbottom(dataset_ntd, wellname, fm)
            df_list.append(df)
        res_df_list = pd.concat(df_list)
        result = pd.concat([dataset_ntd, res_df_list])
        result = result.sort_values(by=['well','top_tst'])
        result_final = result.dropna(subset=['not_htst'], axis=0)

        return result_final
    ntd_val_clean = nothtst_nan_filling(ntd_val, fm)

    def top_phit_bot_clustering(dataset):
        print('Top & bot calculation')
        def top_phit_bot_collection_run(dataset):
            def top_phit_bot_collection(dataset, wellname):
                data = dataset[dataset.well == wellname]
                data['top_htst'] = data['top_tst'] - data['fm_top_tst']
                data['top_htst'].iloc[1:] = data['not_htst'].iloc[:-1]
                data['bot_htst'] = data['not_htst']
                data = data[[   'well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'khtst',
                                'top_htst','htst','bot_htst', 'layer_top_tst', 'layer_bot_tst',
                                'fm_top_tst', 'fm_bot_tst']]
                return data
            df_lst = []
            for wellname in tqdm(dataset.well.unique()):
                res_df = top_phit_bot_collection(dataset, wellname)
                df_lst.append(res_df)
            top_phi_bot_cluster = pd.concat(df_lst).reset_index(drop=True)
            return top_phi_bot_cluster
        top_phi_bot_cluster = top_phit_bot_collection_run(dataset)

        def top_phit_bot_ntg_run(dataset):
            def top_phit_bot_ntg(dataset, wellname):
                ntg = []
                data = dataset[dataset.well == wellname].reset_index(drop=True)
                for ind, row in data.iterrows():
                    if ind == 0:
                        ntg.append(row['htst']/(row['bot_htst'] + row['htst']))
                    if ind != 0:
                        ntg.append(row['htst']/(row['bot_htst'] + row['htst'] + row['top_htst']))
                    if ind == len(data):
                        ntg.append(row['htst']/(row['top_htst'] + row['htst']))
                result = pd.concat([data, pd.DataFrame({'ntg':ntg})], axis=1)
                return result
            df_lst = []
            for wellname in dataset.well.unique():
                df = top_phit_bot_ntg(dataset, wellname)
                df_lst.append(df)
            top_phi_bot_cluster_ntg = pd.concat(df_lst).reset_index(drop=True)
            return top_phi_bot_cluster_ntg
        # top_phi_bot_cluster_ntg = top_phit_bot_ntg_run(top_phi_bot_cluster)
        
        return top_phi_bot_cluster
    ntd_top_phi_bot = top_phit_bot_clustering(ntd_val_clean)

    return ntd_top_phi_bot.reset_index(drop=True)
ntd_top_vsh_bot8_bp = ntd_dataframe_calculation_bp(df_bal8_v4, 'Balakhany VIII', 'NET')
ntd_top_phi_bot8_bp = ntd_dataframe_calculation_bp(df_bal8_v4_flag, 'Balakhany VIII', 'NET')
ntd_top_phi_bot8_bp.to_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot8_bp_v4.csv')

ntd_top_vsh_bot10_bp = ntd_dataframe_calculation_bp(df_bal10_v4, 'Balakhany X', 'NET')
ntd_top_phi_bot10_bp = ntd_dataframe_calculation_bp(df_bal10_v4_flag, 'Balakhany X', 'NET')
ntd_top_phi_bot10_bp.to_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot10_bp_v4.csv')

# Additional calculation

In [190]:
ntd_top_phi_bot8_bp = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot8_bp_v4.csv').drop('Unnamed: 0', axis=1)
ntd_top_phi_bot10_bp = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot10_bp_v4.csv').drop('Unnamed: 0', axis=1)

In [191]:
def histo_tst(dataset):
    dataset = dataset[(dataset.FORMATION != 'Balakhany VIII') & (dataset.FORMATION != 'Balakhany X')]
    tst_interv_fu = dataset.groupby(['well','FORMATION'])['TST'].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index().rename(columns={'TST':'TST_interv_fu'})
    tst_interv = tst_interv_fu.groupby('well')['TST_interv_fu'].sum().reset_index().rename(columns={'TST_interv_fu':'TST_interv'})
    full_v1 = tst_interv_fu.set_index(['well','FORMATION']).join(tst_interv.set_index('well')).reset_index()
    full_v2 = dataset.set_index(['well','FORMATION']).join(full_v1.set_index(['well','FORMATION'])).reset_index()
    return full_v2
full8_v2 = histo_tst(df_bal8_azr_v2)
fulg10_v2 = histo_tst(df_bal10_vshclp2)

In [ ]:
def hist_net_display(well8, well10):
    well_var8 = well8.groupby('well')['NET'].apply(lambda x: x.sum()*0.1).reset_index()
    well_var10 = well10.groupby('well')['NET'].apply(lambda x: x.sum()*0.1).reset_index()
    fig, ax = plt.subplots(1,3, figsize=(20,5))
    sns.histplot(well_var8[well_var8.NET > 0]['NET'], bins=50, kde=True, ax=ax[0], color='blue')
    sns.histplot(well_var10[well_var10.NET > 0]['NET'], bins=50, kde=True, ax=ax[1], color='green')
    sns.histplot(well_var8[well_var8.NET > 0]['NET'], bins=50, kde=True, ax=ax[2], color='blue', label='Bal8')
    sns.histplot(well_var10[well_var10.NET > 0]['NET'], bins=50, kde=True, ax=ax[2], color='green', label='Bal10')
    ax[0].set_title('NET sum distribution Bal8')
    ax[1].set_title('NET sum distribution Bal10')
    ax[2].set_title('NET sum distribution Bal8 & Bal10')
    ax[2].legend()
    ax[0].grid(), ax[1].grid(), ax[2].grid()
    # fig.show()
hist_net_display(full8_v2, fulg10_v2)

In [193]:
bal10_xy = df_bal10_vshclp2.groupby('well')[['Xmean','Ymean', 'field']].first()
ntd_top_phi_bot10_bp_v2 = ntd_top_phi_bot10_bp.set_index('well').join(bal10_xy).reset_index()
ntd_top_phi_bot10_bp_v2 = ntd_top_phi_bot10_bp_v2[ntd_top_phi_bot10_bp_v2.htst !=0]

bal8_xy = df_bal8_azr_v2.groupby('well')[['Xmean','Ymean', 'field']].first()
ntd_top_phi_bot8_bp_v2 = ntd_top_phi_bot8_bp.set_index('well').join(bal8_xy).reset_index()
ntd_top_phi_bot8_bp_v2 = ntd_top_phi_bot8_bp_v2[ntd_top_phi_bot8_bp_v2.htst !=0]

In [194]:
def htst_categorizarions(dataset):
    dataset.loc[(dataset.htst.between(0,1, inclusive='right')), 'htst_range'] = 'Below 1m'
    dataset.loc[(dataset.htst.between(1,2, inclusive='right')), 'htst_range'] = '1-2m'
    dataset.loc[(dataset.htst.between(2,3, inclusive='right')), 'htst_range'] = '2-3m'
    dataset.loc[(dataset.htst.between(3,4, inclusive='right')), 'htst_range'] = '3-4m'
    dataset.loc[(dataset.htst.between(4,5, inclusive='right')), 'htst_range'] = '4-5m'
    dataset.loc[(dataset.htst.between(5,7, inclusive='right')), 'htst_range'] = '5-7m'
    dataset.loc[(dataset.htst >=7), 'htst_range'] = 'Above 7m'
    return dataset

ntd_top_phi_bot8_bp_v2 = htst_categorizarions(ntd_top_phi_bot8_bp_v2)
ntd_top_phi_bot10_bp_v2 = htst_categorizarions(ntd_top_phi_bot10_bp_v2)

In [195]:
ntd_top_phi_bot10_bp_v2_azr = ntd_top_phi_bot10_bp_v2[ntd_top_phi_bot10_bp_v2.field.isin(['CENTRAL AZERI', 'WEST AZERI', 'EAST AZERI'])]
ntd_top_phi_bot10_bp_v2_chi = ntd_top_phi_bot10_bp_v2[ntd_top_phi_bot10_bp_v2.field.isin(['CHIRAG', 'DWG', 'DDGG', 'WEST CHIRAG'])]

In [196]:
def htst_fraction_1m_calculation(dataset):
    htst_count = dataset.groupby(['well','htst_range'])['htst'].count().reset_index().rename(columns={'htst':'htst_count'})
    htst_sum = htst_count.groupby('well')['htst_count'].sum().reset_index().rename(columns={'htst_count':'htst_sum'})
    htst_full = htst_count.set_index('well').join(htst_sum.set_index('well')).reset_index()
    htst_full['percent'] = (htst_full['htst_count']/htst_full['htst_sum']*100).astype(int)
    return htst_full
htst_full8 = htst_fraction_1m_calculation(ntd_top_phi_bot8_bp_v2)
htst_full10_azr = htst_fraction_1m_calculation(ntd_top_phi_bot10_bp_v2_azr)
htst_full10_chi = htst_fraction_1m_calculation(ntd_top_phi_bot10_bp_v2_chi)

In [ ]:
def display_categorized_htst():
    fig, ax = plt.subplots(1,2, figsize=(15,5))
    sns.histplot(htst_full10_azr[htst_full10_azr.htst_range == 'Below 1m'], x='percent', 
                 bins=50, kde=True, color='#0da1ce', ax=ax[0], zorder = 2)
    sns.histplot(htst_full10_chi[htst_full10_chi.htst_range == 'Below 1m'], x='percent', 
                 bins=50, kde=True, color='#1436cc', ax=ax[1], zorder = 2)
    ax[0].set_title('Fraction of layers below 1m per well Bal10 Azeri')
    ax[1].set_title('Fraction of layers below 1m per well Bal10 CDWG')
    ax[0].set_xlabel('%')
    ax[1].set_xlabel('%')
    ax[0].grid(True, zorder=1)
    ax[1].grid(True, zorder=1);
display_categorized_htst()

In [198]:
ntd_top_phi_bot8_bp_v2.to_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot8_bp_v2.csv')
ntd_top_phi_bot10_bp_v2.to_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot10_bp_v2.csv')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,5))
sns.histplot(ntd_top_phi_bot8_bp[ntd_top_phi_bot8_bp.htst > 1]['htst'], bins=50, kde=True, ax=ax[0], color='blue')
sns.histplot(ntd_top_phi_bot10_bp[ntd_top_phi_bot10_bp.htst > 1]['htst'], bins=50, kde=True, ax=ax[1], color='green')

ax[0].set_title('Layers Htst>1m distribution Bal8 ')
ax[1].set_title('Layers Htst>1m distribution Bal10')
ax[0].grid() 
ax[1].grid()

# Logs-df enrichment with ntd-df

In [142]:
logs8 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v4.csv')
logs8.columns = logs8.columns.str.lower()
ntd8 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot8_bp_v4.csv').drop('Unnamed: 0', axis=1)
ntd8.columns = ntd8.columns.str.lower()
logs10 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_v4.csv')
logs10.columns = logs10.columns.str.lower()
ntd10 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot10_bp_v4.csv').drop('Unnamed: 0', axis=1)
ntd10.columns = ntd10.columns.str.lower()

In [143]:
def logs_join_ntd(dataset_ntd, dataset_logs):
    well_lst = []
    for wellname in dataset_ntd.well.unique():
        data = dataset_ntd[dataset_ntd.well == wellname]
        well_layers_lst = []
        for idx, var in data.iterrows():
            start_depth = var['layer_top_tst']
            end_depth = var['layer_bot_tst']
            new_tst = np.arange(start_depth, end_depth, 0.1)
            new_well = [wellname for i in range(len(new_tst))]
            new_htst = [var['htst'] for i in range(len(new_tst))]
            df = pd.DataFrame({'well':new_well,'tst':new_tst, 'htst':new_htst})
            well_layers_lst.append(df)
        well_ntd_point = pd.concat(well_layers_lst).reset_index(drop=True)
        well_lst.append(well_ntd_point)
    result = pd.concat(well_lst)
    dataset_logs['tst'] = dataset_logs['tst'].round(1)
    result['tst'] = result['tst'].round(1)
    log_ntd = dataset_logs.set_index(['well','tst']).join(result.set_index(['well','tst'])).reset_index()
    log_ntd['htst'] = log_ntd['htst'].fillna(0)
    log_ntd = log_ntd.rename(columns={'htst':'htst_ntd'})
    log_ntd.columns = log_ntd.columns.str.lower()
    return log_ntd
logs8_ntd = logs_join_ntd(ntd8, logs8)
logs10_ntd = logs_join_ntd(ntd10, logs10)
logs8_ntd.to_csv(r'C:\jupyter\SPP\inputoutput\general_logs\logs8_ntd_v4.csv', index=False)
logs10_ntd.to_csv(r'C:\jupyter\SPP\inputoutput\general_logs\logs10_ntd_v4.csv', index=False)

# Experiments with data

In [ ]:
def well_gr_smooth_display_v1(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['VSH_smooth']
    grn_init = well_sel['VSH']
    phit = well_sel['PHIT']
    net = well_sel[var]
    net_smooth = well_sel['NET_smooth']*100
    # net_smooth = net
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=2, subplot_titles=('VSH', 'PHIT'))
    fig.add_trace(go.Scatter(x=grn_init, y=depth, mode='lines', name='VSH', 
                             line=dict(color='green')), row=1, col=1)
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='VSH', 
                             line=dict(color='yellow')), row=1, col=1)  
    fig.update_xaxes(range = [0,1])
    fig.add_trace(go.Scatter(x=net_smooth, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 1], row=1, col=1)

    # fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            # line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=900, width=400,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50))
    fig.show()
well_gr_smooth_display_v1(df_bal8_azr_v2, 'Balakhany VIII', 'NET_orig', 'B16Z')

In [ ]:
# B30, B36 - 0 Fluid (bal8)
# D20X - 5, 7 Fluid (bal10)
# E17, J07, J14, J17, J21 - Fluid 7 (bal10)
# B18X, C03, C05, C08, C15Z, C19, C22, C24, D20Z - Fluid 7 (bal8)

In [ ]:
fluids = pd.read_csv(r'C:\jupyter\SPP\input\fluidcode_techlog.csv')[['wellName', 'MD', 'FLUIDS']][1:]
fluids = fluids.rename(columns={'wellName':'well'})
print(fluids.columns)

In [153]:
fluids = fluids[fluids['FLUIDS'] != '-9999']
fluids = fluids[fluids['FLUIDS'] != -9999.]
fluids = fluids.dropna()
fluids = fluids.astype({'FLUIDS':'int', 'MD':'float', 'well':'string'})
fluids['MD'] = fluids['MD'].round(1)

In [151]:
df8 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v4.csv').drop('Unnamed: 0', axis=1)
df10 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_v4.csv').drop('Unnamed: 0', axis=1)
df = pd.concat([df8, df10])
df_fluid = df.set_index(['well','MD']).join(
                        fluids.set_index(['well','MD'])).reset_index().astype(
                        {'well':'string','FORMATION_up':'string', 'field':'string'}).drop('V', axis=1)

In [166]:
df_fluid['FLUIDS'] = df_fluid['FLUIDS'].fillna(0)
color_dict = {0:'yellow', 1.: 'red', 2.: 'green', 3.: 'blue', 4.:'lightgreen', 
                5.: 'purple', 6.: 'darkgreen', 7.: 'orange', 8.: 'gray'}
df_fluid['fluid_color'] = df_fluid['FLUIDS'].map(color_dict)

In [ ]:
logs_gas_fluids = df_fluid[(df_fluid.FLUIDS == 1) | (df_fluid.FLUIDS == 2) | (df_fluid.FLUIDS == 3)]
# logs_gas_fluids = logs_gas_fluids[logs_gas_fluids.PHIT > 0]
logs_gas_details = df_fluid[(df_fluid.FLUIDS == 1) | (df_fluid.FLUIDS == 4) | (df_fluid.FLUIDS == 5) | (df_fluid.FLUIDS == 7)]

fig, ax = plt.subplots(1,2, figsize=(12,5))
custom_palette_gas = {1.:'red',2.:'green',3.:'blue'}
sns.histplot(   data=logs_gas_fluids[logs_gas_fluids.FORMATION_up == 'Balakhany VIII'], 
                x = 'PHIT', hue='FLUIDS', bins=60, palette=custom_palette_gas, ax=ax[0])
ax[0].set_title('Balakhany VIII')
sns.histplot(   data=logs_gas_fluids[logs_gas_fluids.FORMATION_up == 'Balakhany X'], 
                x = 'PHIT', hue='FLUIDS', bins=60, palette=custom_palette_gas, ax=ax[1])
ax[1].set_title('Balakhany X')

fig, ax = plt.subplots(1,2, figsize=(12,5))
custom_palette_gas_det = {1.:'red',4.:'lightgreen', 5.:'purple', 7.:'orange'}
sns.histplot(   data=logs_gas_details[logs_gas_details.FORMATION_up == 'Balakhany VIII'], 
                x = 'PHIT', hue='FLUIDS', bins=60, palette=custom_palette_gas_det, ax=ax[0])
sns.histplot(   data=logs_gas_details[logs_gas_details.FORMATION_up == 'Balakhany X'], 
                x = 'PHIT', hue='FLUIDS', bins=60, palette=custom_palette_gas_det, multiple = 'stack', ax=ax[1])

In [ ]:
def well_display_logs_fluid(dataset, wellname, fmname, var1, var2, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    data_net_khtst = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname) & 
                             ((dataset.FORMATION != 'Balakhany VIII') & (dataset.FORMATION != 'Balakhany X'))]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data[var1]*100
    net_init_calc = data_net_khtst[var1]
    net = data[var2]*100
    net_calc = data_net_khtst[var2]
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    khtst = data['KHtst']
    fluid = data['FLUIDS']
    fluid_color = data['fluid_color']

    var1_sum = (net_init_calc.sum()*0.1).round(0)
    var2_sum = (net_calc.sum()*0.1).round(0)
    tst_interval = round(depth.max() - depth.min(),0)
    height = int(tst_interval/15)
    khtst_top = int(data_net_khtst['KHtst'].iloc[0])

    fig, ax = plt.subplots(1,4, figsize=(7,height), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # ax[0].set_title((var1 + ':' + str(var1_sum) + 'm'), fontsize=10)
    ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    # ax[2].set_title((var2 + ':' + str(var2_sum) + 'm'), fontsize=10)
    ax[2].set_title(var2 + ':' + str(var2_sum), fontsize=10)
    # ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth, net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    twin4 = ax[3].twiny()
    twin4.plot(khtst, depth, color='black', alpha=1)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    twin5 = ax[3].twiny()
    twin5.plot(fluid, depth, color='black')
    # twin5.fill_betweenx(fluid, depth, color=fluid_color, alpha=0.33)
    twin5.set_xlim(0, 8)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' 
                + ref_depth +':' + str(tst_interval) + 
                ' KHtst:' + str(khtst_top), fontsize=12)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png', dpi=300)
    else:
        pass
    plt.show();

well_display_logs_fluid(df_fluid, 'B16Z', 'Balakhany VIII', 'NET_VSH', 'NET', 'TST', 5, 'dontprint', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal8\\')